# Smart Home Automation

**Team Number:** 1

**Team Members:**

*   Anirudh Jayan
*   Abhinav Variyath
*   Tara Samiksha
*   Sarvesh Ram Kumar
*   Aravind S Harilal

## Introduction & Problem Statement

**Goal:** Create a Smart Home Automation System to manage electronic devices, optimize electricity consumption, and reduce energy wastage.

**Key Features:**

*   Device management using a **Priority Queue**
*   Automation rule implementation with a **Linked List**
*   Energy-efficient device coordination

## Linked List: Structured Rule Automation System

### What is a Linked List?

A linear data structure where elements (nodes) are linked sequentially. It excels at dynamic insertion and deletion of elements.

### Why Use a Linked List for Automation Rules?

*   **Ordered Rule Execution:**  Automation rules often need to be executed in a specific sequence.
*   **Dynamic Rule Sets:** Easily add, remove, or modify rules without complex restructuring.

### Rule Execution Process

1. Rules are parsed from user input (or external sources) and converted into `Rule` objects.
2. `Rule` objects are added to the `LinkedList`.

In [12]:
import threading 
class Node:  # For creation of a node. Each node contains the data and the address of the next node.
    def __init__(self, val):
        self.val = val
        self.next = None

class LinkedList:  # Linked list implementation.
    def __init__(self):
        self.head = None
        self.size = 0
        self.lock = threading.Lock()

    def add_front(self, val):  # Adding a node in the front of the linked list.
        with self.lock:
            new_node = Node(val)
            new_node.next = self.head
            self.head = new_node
            self.size += 1

    def add_end(self, val):  # Adding a node at the end of the linked list.
        new_node = Node(val)

        with self.lock:
            if not self.head:
                self.head = new_node

            else:
                temp = self.head
                while temp.next:
                    temp = temp.next
                temp.next = new_node
            self.size += 1

    def peek(self):  # Gives us the data in the very first node without popping it.
        with self.lock:
            return self.head.val if self.head else None

    def peek_end(self):  # Gives us the data at the last node without popping it.
        with self.lock:
            if not self.head:
                return None
            temp = self.head
            while temp.next:
                temp = temp.next
            return temp.val

    def get_size(self):  # Gives us the size of the linked list.

        with self.lock:
            return self.size

    def print_list(self):  # Displays the entire list.

        with self.lock:
            temp = self.head
            while temp:
                print(temp.val, end=" -> ")
                temp = temp.next
            print("None")

    def clear(self):  # Clears the entire list.

        with self.lock:
            self.head = None
            self.size = 0

    def is_empty(self):  # Checks if the list is empty.

        with self.lock:
            return self.size == 0

    def remove_end(self):  # Peek and pop the last element.
        with self.lock:
            if not self.head:
                return None  # List is empty

            if not self.head.next:
                val = self.head.val  # Only one element in the list
                self.head = None
                self.size -= 1
                return val

            temp = self.head
            while temp.next and temp.next.next:
                temp = temp.next

            val = temp.next.val  # Peek the last element
            temp.next = None  # Remove it
            self.size -= 1
            return val
        
    
    def remove_position(self, position):

        if position < 0 or position >= self.size:
            return None  
        
        if position == 0:
            return self.remove_beginning()
            
        current = self.head
        for i in range(position - 1):
            current = current.next
        val = current.next.val
        current.next = current.next.next
        
        self.size -= 1
        
        return val

    def remove_beginning(self):  # Peeks and pops the first element
        with self.lock:
            if not self.head:
                return None  # List is empty
            val = self.head.val  # Peek the first element
            self.head = self.head.next  # Remove it
            self.size -= 1
            return val

    def _make_list(self):
        temp = self.head
        array_list = []

        while temp:
            array_list.append(temp.val)
            temp = temp.next

        return array_list

ll = LinkedList()
ll.add_end(10)
ll.add_end(20)
ll.add_front(5)
ll.print_list()
print(ll.peek())
print(ll.get_size())
print(ll.remove_end())
print(ll.remove_end())
ll.add_end(20)
print(ll.peek())
print(ll.remove_beginning())

5 -> 10 -> 20 -> None
5
3
20
10
5
5


## Priority Queue: Intelligent & Prioritized Device Task Handling

### What is a Priority Queue?

A data structure that orders elements based on their priority, ensuring that higher priority elements are processed first.

### Why Use a Priority Queue in a Smart Home?

*   **Real-world Prioritization:**  Different devices have varying levels of importance.
    *   **Examples:**`
        *   Security Alarms (High Priority)
        *   Lighting (Medium Priority)
        *   Decorative Displays (Low Priority)
*   **Resource Allocation:** Efficiently manages system resources by prioritizing critical tasks.

### Priority Calculation

The priority of a device is determined by the following formula:
Priority = Device Type Priority + Device Group Priority + (Location Occupancy * Weight)


**Example:** A security camera in an occupied living room will have a higher priority than a decorative light in an empty bedroom.

In [13]:
import threading

class Task:  # Tasks with a set priority
    def __init__(self, task, priority):
        self.task = task
        self.priority = priority

    def __repr__(self):
        return f"Task(priority={self.priority}, task={self.task})"


class PriorityQueue:
    def __init__(self, task=None):
        self.queue = LinkedList()  # Use Linked List instead of Python List
        self.lock = threading.Lock()

        if task:
            self.enqueue(task) 

    def enqueue(self, newTask: Task):
        with self.lock:
            self._enqueue_internal(newTask)

    def _enqueue_internal(self, newTask: Task):
        """Internal version of enqueue that doesn't acquire the lock"""
        if self.queue.is_empty():
            self.queue.add_end(newTask)
            return

        current = self.queue.head
        prev = None

        while current:
            if newTask.priority < current.val.priority:
                if prev is None:
                    self.queue.add_front(newTask)
                else:
                    new_node = Node(newTask)
                    new_node.next = current
                    prev.next = new_node
                    self.queue.size += 1
                return
            prev = current
            current = current.next

        self.queue.add_end(newTask)

    def dequeue(self):  # Dequeuing the element with the most priority.
        with self.lock:
            if self.queue.is_empty():
                return None 
            return self.queue.remove_beginning()

    def peek(self):  # Returns the element with the most priority
        with self.lock:
            if self.queue.is_empty():
                return None
            return self.queue.peek()

    def print(self):  # Prints the queue with the priorities too.
        with self.lock:
            current = self.queue.head
            while current:
                print(f"Priority: {current.val.priority}, Task: {current.val.task}")
                current = current.next

    def is_empty(self):  # Checks if the queue is empty.
        with self.lock:
            return self.queue.is_empty()

    def size(self):  # Checks the size of the queue.
        with self.lock:
            return self.queue.get_size()

    def clear(self):  # clears the queue.
        with self.lock:
            self.queue.clear()

    def contains(self, task):  # Checks if the task is in the queue.
        with self.lock:
            return self._contains_internal(task)

    def _contains_internal(self, task):
        """Internal version of contains that doesn't acquire the lock"""
        current = self.queue.head
        while current:
            if current.val == task:
                return True
            current = current.next
        return False

    def get_priority(self, task_content):  # Gets the priority of a given task.
        with self.lock:
            return self._get_priority_internal(task_content)

    def _get_priority_internal(self, task_content):
        """Internal version of get_priority that doesn't acquire the lock"""
        current = self.queue.head
        while current:
            if current.val.task == task_content:
                return current.val.priority
            current = current.next
        return -1

    def remove_task(self, task):
        with self.lock:
            task_obj = self._get_task_internal(task)
            if task_obj:
                index = self._get_task_index_internal(task)
                if index != -1:
                    self.queue.remove_position(index)
                    return True
            return False

    def get_task(self, task):  # Get Task Using the task's name
        with self.lock:
            return self._get_task_internal(task)
    
    def _get_task_internal(self, task):
        """Internal version of get_task that doesn't acquire the lock"""
        current = self.queue.head
        while current:
            if current.val.task == task:
                return current.val
            current = current.next
        return None

    def get_task_index(self, task):  # Get Task Using the task's name
        with self.lock:
            return self._get_task_index_internal(task)

    def _get_task_index_internal(self, task):
        """Internal version of get_task_index that doesn't acquire the lock"""
        current = self.queue.head
        index = 0
        while current:
            if current.val.task == task:
                return index
            current = current.next
            index += 1
        return -1

    def update_priority(self, task_name, priority):
        with self.lock:
            index = self._get_task_index_internal(task_name)
            if index != -1:
                self.queue.remove_position(index)
                self._enqueue_internal(Task(task_name, priority))
                return True
            return False

# Test code
if __name__ == "__main__":
    pq = PriorityQueue()
    pq.enqueue(Task("Do laundry", 3))
    pq.enqueue(Task("Finish report", 1))
    pq.enqueue(Task("Buy groceries", 2))

    print("\nQueue after enqueuing:")
    pq.print()

    print("\nDequeuing highest priority task")
    pq.dequeue()
    pq.print()

    print("\nPeeking at the highest priority task:")
    print(pq.peek())

    print(pq.get_priority("Buy groceries"))
    pq.print()

    print("\nUpdating priority of 'Buy groceries' to 0:")
    pq.update_priority("Buy groceries", 0)
    pq.print()
    
    print("\nRemoving 'Do laundry':")
    pq.remove_task("Do laundry")
    print("####################")
    pq.print()


Queue after enqueuing:
Priority: 1, Task: Finish report
Priority: 2, Task: Buy groceries
Priority: 3, Task: Do laundry

Dequeuing highest priority task
Priority: 2, Task: Buy groceries
Priority: 3, Task: Do laundry

Peeking at the highest priority task:
Task(priority=2, task=Buy groceries)
2
Priority: 2, Task: Buy groceries
Priority: 3, Task: Do laundry

Updating priority of 'Buy groceries' to 0:
Priority: 0, Task: Buy groceries
Priority: 3, Task: Do laundry

Removing 'Do laundry':
####################
Priority: 0, Task: Buy groceries


## Energy-Efficient Device Coordination

This system employs a two-pronged approach to optimize energy consumption:

### 1. Rule-Based Energy Optimization (Linked List)

*   Users can define energy-saving rules, such as:
    *   "If the time is after 11 PM and motion is not detected in the living room for 15 minutes, turn off the living room lights."
*   The `LinkedList` ensures that these rules are executed efficiently and in the correct order.

In [14]:
class Rule:
    def __init__(self, device_id= -1, flip_state = False, turn_on = False, turn_off = False, set_power_level =False, power_level = 0, group_name = "",
                turn_group_off = False, turn_group_on= False, type_name= "", turn_type_off= False, turn_type_on= False, location_name= "", turn_location_off= False, turn_location_on= False):
        self.__deviceId = device_id
        self.__flipState = flip_state
        self.__turnOn = turn_on
        self.__turnOff = turn_off
        self.__setPowerLevel = set_power_level
        self.__powerLevel = power_level
        self.__groupName = group_name
        self.__turnGroupOff = turn_group_off
        self.__turnGroupOn = turn_group_on
        self.__typeName = type_name
        self.__turnTypeOff = turn_type_off
        self.__turnTypeOn = turn_type_on
        self.__locationName = location_name
        self.__turnLocationOff = turn_location_off
        self.__turnLocationOn = turn_location_on

    def get_device_id(self):
        return self.__deviceId

    def set_device_id(self, device_id):
        self.__deviceId = device_id

    def get_flip_state(self):
        return self.__flipState

    def set_flip_state(self, flip_state):
        self.__flipState = flip_state

    def get_turn_on(self):
        return self.__turnOn

    def set_turn_on(self, turn_on):
        self.__turnOn = turn_on

    def get_turn_off(self):
        return self.__turnOff

    def set_turn_off(self, turn_off):
        self.__turnOff = turn_off

    def get_set_power_level(self):
        return self.__setPowerLevel

    def set_set_power_level(self, set_power_level):
        self.__setPowerLevel = set_power_level

    def get_power_level(self):
        return self.__powerLevel

    def set_power_level(self, power_level):
        self.__powerLevel = power_level

    def get_group_name(self):
        return self.__groupName

    def set_group_name(self, group_name):
        self.__groupName = group_name

    def get_turn_group_off(self):
        return self.__turnGroupOff

    def set_turn_group_off(self, turn_group_off):
        self.__turnGroupOff = turn_group_off

    def get_turn_group_on(self):
        return self.__turnGroupOn

    def set_turn_group_on(self, turn_group_on):
        self.__turnGroupOn = turn_group_on

    def get_type_name(self):
        return self.__typeName

    def set_type_name(self, type_name):
        self.__typeName = type_name

    def get_turn_type_off(self):
        return self.__turnTypeOff

    def set_turn_type_off(self, turn_type_off):
        self.__turnTypeOff = turn_type_off

    def get_turn_type_on(self):
        return self.__turnTypeOn

    def set_turn_type_on(self, turn_type_on):
        self.__turnTypeOn = turn_type_on

    def get_location_name(self):
        return self.__locationName

    def set_location_name(self, location_name):
        self.__locationName = location_name

    def get_turn_location_off(self):
        return self.__turnLocationOff

    def set_turn_location_off(self, turn_location_off):
        self.__turnLocationOff = turn_location_off

    def get_turn_location_on(self):
        return self.__turnLocationOn

    def set_turn_location_on(self, turn_location_on):
        self.__turnLocationOn = turn_location_on

    def __str__(self):
        return f"Rule(deviceId={self.__deviceId}, flipState={self.__flipState}, turnOn={self.__turnOn}, turnOff={self.__turnOff}, setPowerLevel={self.__setPowerLevel}, powerLevel={self.__powerLevel}, groupName={self.__groupName}, turnGroupOff={self.__turnGroupOff}, turnGroupOn={self.__turnGroupOn}, typeName={self.__typeName}, turnTypeOff={self.__turnTypeOff}, turnTypeOn={self.__turnTypeOn}, locationName={self.__locationName}, turnLocationOff={self.__turnLocationOff}, turnLocationOn={self.__turnLocationOn})"

    def to_dict(self):
        return {
            "device_id": self.__deviceId,
            "flip_state": self.__flipState,
            "turn_on": self.__turnOn,
            "turn_off": self.__turnOff,
            "set_power_level_flag": self.__setPowerLevel,
            "power_level_value": self.__powerLevel,
            "group_name": self.__groupName,
            "turn_group_off": self.__turnGroupOff,
            "turn_group_on": self.__turnGroupOn,
            "type_name": self.__typeName,
            "turn_type_off": self.__turnTypeOff,
            "turn_type_on": self.__turnTypeOn,
            "location_name": self.__locationName,
            "turn_location_off": self.__turnLocationOff,
            "turn_location_on": self.__turnLocationOn
        }
    
#TESTING
rule = Rule("123", True, False, True, False, 0, "Living Room", False, True, "Light", False, True, "Home", False, True)
print(rule)

Rule(deviceId=123, flipState=True, turnOn=False, turnOff=True, setPowerLevel=False, powerLevel=0, groupName=Living Room, turnGroupOff=False, turnGroupOn=True, typeName=Light, turnTypeOff=False, turnTypeOn=True, locationName=Home, turnLocationOff=False, turnLocationOn=True)


### 2. Priority Queue Driven Power Management

*   **Monitoring:** The system continuously monitors overall power consumption against a predefined threshold.
*   **Threshold Exceeded:** If the power consumption threshold is exceeded, the `Priority Queue` identifies devices with lower priority.
*   **Dynamic Rule Generation:** The system automatically creates and adds rules to the `LinkedList` to reduce power consumption for lower-priority devices. These rules might involve:
    *   Turning off the device.
    *   Reducing the power level (e.g., dimming lights).
*   **Dynamic Adaptation:** The system continuously adapts to power usage, adjusting device states based on priority and rules to maintain energy efficiency in real-time.


In [15]:
from enum import Enum
import time

class Device:
    def __init__(self, device_id: int, device_name: str, device_type, location, device_group, battery_level, max_battery_capacity,current_battery_capacity, is_on_battery, is_turned_on, base_power_consumption, power_level,turned_on_time, is_interacted):
        self.__device_id = device_id
        self.__device_name = device_name
        self.__device_type = device_type
        self.__location = location
        self.__device_group = device_group
        self.__battery_level = battery_level
        self.__max_battery_capacity = max_battery_capacity
        self.__current_battery_capacity = current_battery_capacity
        self.__is_on_battery = is_on_battery
        self.__is_turned_on = is_turned_on
        self.__base_power_consumption = base_power_consumption
        self.__power_level = power_level
        self.__turned_on_time = turned_on_time
        self.__is_interacted = is_interacted

    def flip_interaction_state(self):
        self.__is_interacted = not self.__is_interacted

    def get_interaction_state(self) -> bool:
        return self.__is_interacted

    def get_minutes_since_turned_on(self):
        if self.__turned_on_time:
            return int((int(time.time()) - self.__turned_on_time) // 60)
        return 0

    def set_turned_on(self, status: bool):
        self.__is_turned_on = status

    def is_turned_on(self) -> bool:
        return self.__is_turned_on

    def set_battery_level(self, level: float):
        self.__battery_level = level

    def get_battery_level(self) -> float:
        return self.__battery_level

    def set_base_power_consumption(self, consumption: float):
        self.__base_power_consumption = consumption

    def get_base_power_consumption(self) -> float:
        return self.__base_power_consumption

    def set_battery_capacity(self, capacity: int):
        self.__max_battery_capacity = capacity

    def get_battery_capacity(self) -> int:
        return self.__max_battery_capacity

    def get_device_id(self) -> int:
        return self.__device_id

    def get_device_name(self) -> str:
        return self.__device_name

    def set_device_name(self, name: str):
        self.__device_name = name

    def get_device_type(self):
        return self.__device_type

    def set_device_type(self, type_: str):
        self.__device_type = type_

    def get_location(self):
        return self.__location

    def set_location(self, location: str):
        self.__location = location

    def get_device_group(self):
        return self.__device_group

    def set_device_group(self, group: str):
        self.__device_group = group

    def get_power_level(self) -> int:
        return self.__power_level

    def set_power_level(self, level: int):
        self.__power_level = level

    def is_on_battery_power(self) -> bool:
        return self.__is_on_battery

    def set_on_battery(self, status: bool):
        self.__is_on_battery = status

    def get_current_battery_capacity(self) -> float:
        return self.__current_battery_capacity

    def set_current_battery_capacity(self, capacity: float):
        self.__current_battery_capacity = capacity

    def set_turned_on_time(self, time: int):
        self.__turned_on_time = time

    def get_turned_on_time(self) -> int:
        return self.__turned_on_time

    def __str__(self):
        return (f"Device ID: {self.get_device_id()}\n"
                f"Device Name: {self.get_device_name()}\n"
                f"Device Type: {self.get_device_type()}\n"
                f"Device Group: {self.get_device_group()}\n"
                f"Location: {self.get_location()}\n"
                f"Power Status: {'On' if self.is_turned_on() else 'Off'}\n"
                f"Battery Level: {self.get_battery_level()}\n"
                f"Power Consumption: {self.get_base_power_consumption()} W\n"
                f"Power Level: {self.get_power_level()}\n")
    def to_dict(self):
    
        def safe_json_val(val):
            if isinstance(val, Enum):
                if isinstance(val, DeviceLocationEnum):
                    return val.value
                else:
                    return val.name 
            elif val is None:
                    return None 
            else:
                    return val

        return {
            'device_id': self.get_device_id(),
            'device_name': self.get_device_name(),
            'device_type': safe_json_val(self.get_device_type()),
            'location': safe_json_val(self.get_location()),
            'device_group': safe_json_val(self.get_device_group()),
            'battery_level': self.get_battery_level(),
            'max_battery_capacity': self.get_battery_capacity(),
            'current_battery_capacity': self.get_current_battery_capacity(),
            'is_on_battery': self.is_on_battery_power(),
            'is_turned_on': self.is_turned_on(),
            'base_power_consumption': self.get_base_power_consumption(),
            'power_level': self.get_power_level(),
            'turned_on_time': self.get_turned_on_time(), 
            'minutes_since_turned_on': self.get_minutes_since_turned_on(),
            'is_interacted': self.get_interaction_state()
        }

#Test

device1 = Device(
    device_id=1,
    device_name="Light",
    device_type="Decorative",
    location="Living Room",
    device_group="LIGHTS",
    battery_level=85.0,
    max_battery_capacity=100,
    current_battery_capacity=85.0,
    is_on_battery=False,
    is_turned_on=True,
    base_power_consumption=10.0,
    power_level=5,
    turned_on_time=int(time.time()),
    is_interacted=False,

)

print(device1)

device1.set_battery_level(90.0)
device1.set_power_level(8)
device1.flip_interaction_state()

print(f"Battery Level after update: {device1.get_battery_level()}")
print(f"Power Level after update: {device1.get_power_level()}")
print(f"Interaction State: {device1.get_interaction_state()}")
print(f"Minutes since turned on: {device1.get_minutes_since_turned_on()} min")


Device ID: 1
Device Name: Light
Device Type: Decorative
Device Group: LIGHTS
Location: Living Room
Power Status: On
Battery Level: 85.0
Power Consumption: 10.0 W
Power Level: 5

Battery Level after update: 90.0
Power Level after update: 8
Interaction State: True
Minutes since turned on: 0 min


In [16]:
class AirConditioner(Device):
    def __init__(self,
                device_id: int,
                device_name: str,
                device_type,
                location,   
                device_group,
                is_turned_on: bool,
                battery_level: float,       
                base_power_consumption: float,
                max_battery_capacity: int,
                current_battery_capacity: int, 
                power_level: int,
                is_on_battery: bool,           
                turned_on_time: int,           
                is_interacted: bool = False,   
                mode: bool = True             
            ):

        super().__init__(
            device_id=device_id,
            device_name=device_name,
            device_type=device_type,
            location=location,            
            device_group=device_group,      
            is_turned_on=is_turned_on,
            battery_level=battery_level,
            base_power_consumption=base_power_consumption, 
            max_battery_capacity=max_battery_capacity,
            current_battery_capacity=current_battery_capacity, 
            power_level=power_level,
            is_on_battery=is_on_battery,         
            turned_on_time=turned_on_time,        
            is_interacted=is_interacted          
        )

        self._mode = mode 
        self.simulation_temp_change_time = time.time()

    def get_mode(self) -> bool:
        return self._mode

    def set_mode(self, mode: bool):
        self._mode = mode

    def toggle_mode(self):
        self._mode = not self._mode

    def get_simulation_temp_change_time(self) -> int:
        return self.simulation_temp_change_time

    def set_simulation_temp_change_time(self, simulation_temp_change_time: int):
        self.simulation_temp_change_time = simulation_temp_change_time

    def get_minutes_since_temp_change(self) -> int:
        return int((int(time.time()) - self.simulation_temp_change_time) // 60)

    def __str__(self) -> str:
        return super().__str__() + " Mode: " + ("Cooling" if self.get_mode() else "Heating")


### 3. Enum Based Priority Calculation

*   **Easy Modification:** Modifying priority values is as simple as changing one number
*   **Fast and Easy Calculation:** All priority values can be accessed and calculated very easily
*   **Ease of Use** User does not have to manually include priority values for every single device added

In [17]:
from enum import Enum


class DeviceGroupEnum(Enum):
    LIGHTS = 10
    FANS = 9
    ALARMS = 15
    CAMERAS = 14
    AIRCONDITIONERS = 8
    HEATERS = 8
    APPLIANCES = 6
    GARDENING = 3
    ENTERTAINMENT = 2
    CLEANING = 5
    LAUNDRY = 4
    WEARABLES = 7
    BATHROOM = 12
    OTHERS = 1

    def get_priority(self):
        return self.value


class DeviceGroup:
    def __init__(self, group_name: str):
        self.group_name = group_name
        self.devices = []

    def add_device(self, device: Device):
        self.devices.append(device)

    def remove_device(self, device: Device):
        if device in self.devices:
            self.devices.remove(device)

    def get_devices(self):
        return self.devices

    def turn_off_all_devices(self):
        for device in self.devices:
            device.set_turned_on(False)

    def turn_on_all_devices(self):
        for device in self.devices:
            device.set_turned_on(True)

    def get_device_by_name(self, name: str):
        name_lower = name.lower()
        return next((device for device in self.devices if name_lower in device.get_device_name().lower()), None)

    def get_device_by_id(self, device_id: int):
        return next((device for device in self.devices if device.get_device_id() == device_id), None)

    def get_group_name(self):
        return self.group_name

In [18]:
from enum import Enum


class DeviceTypeEnum(Enum):
    DECORATIVE = 1,
    HEALTH = 15,
    ENTERTAINMENT = 3,
    SECURITY = 20,
    PERSONALCARE = 7,
    CONNECTIVITY = 10,
    COOKING = 12,
    LUXURY = 2,
    OFFICE = 10,
    OTHERS = 5

    def get_priority(self):
        return self.value


class DeviceType:
    def __init__(self, typeName: str):
        self.typeName = typeName
        self.devices = []

    def add_device(self, device: Device):
        self.devices.append(device)

    def remove_device(self, device: Device):
        if device in self.devices:
            self.devices.remove(device)

    def get_devices(self):
        return self.devices

    def turn_off_all_devices(self):
        for device in self.devices:
            device.set_turned_on(False)

    def turn_on_all_devices(self):
        for device in self.devices:
            device.set_turned_on(True)

    def get_device_by_name(self, name: str):
        name_lower = name.lower()
        return next((device for device in self.devices if name_lower in device.get_device_name().lower()), None)

    def get_device_by_id(self, device_id: int):
        return next((device for device in self.devices if device.get_device_id() == device_id), None)



In [19]:
from enum import Enum


class DeviceLocationEnum(Enum):
    LIVINGROOM = "Living Room"
    BEDROOM = "Bedroom"
    BEDROOM2 = "Bedroom 2"
    BEDROOM3 = "Bedroom 3"
    BEDROOM4 = "Bedroom 4"
    GARDEN = "Garden"
    OFFICE = "Office"
    ENTRANCE = "Entrance"
    KITCHEN = "Kitchen"
    BATHROOM = "Bathroom"
    BATHROOM2 = "Bathroom 2"
    BATHROOM3 = "Bathroom 3"
    OTHERS = "Others"


class DeviceLocation:
    def __init__(self, location: str):
        self.location = location
        self.devices = []
        self.people = 0
        self.temperature = 0.0

    def add_device(self, device: Device):

        self.devices.append(device)

    def remove_device(self, device: Device):

        if device in self.devices:
            self.devices.remove(device)

    def get_devices(self):
        return self.devices

    def get_people(self):
        return self.people

    def set_people(self, people: int):
        self.people = people

    def add_people(self, people: int):
        self.people += people

    def remove_people(self, people: int):
        self.people = max(0, self.people - people)

    def turn_off_all_devices(self):

        for device in self.devices:
            device.set_turned_on(False)

    def turn_on_all_devices(self):

        for device in self.devices:
            device.set_turned_on(True)

    def get_temperature(self):
        return self.temperature

    def set_temperature(self, temperature: float):
        self.temperature = temperature

    def get_device_by_name(self, name: str):

        name_lower = name.lower()
        return next((device for device in self.devices if name_lower in device.get_device_name().lower()), None)

    def get_device_by_id(self, device_id: int):

        return next((device for device in self.devices if device.get_device_id() == device_id), None)

    def __str__(self):
        return f"Location: {self.location}"

### 4. Logging system

*   **Log Monitoring:** Displays system logs for transparency and debugging, including:
    *   Info logs
    *   Warning logs
    *   Severe logs
    *   Power logs
    *   Battery logs

In [20]:
import logging


class LogTask:
    LEVEL_LIST = [
        logging.ERROR,
        logging.CRITICAL,
        logging.WARNING,
        logging.INFO
    ]

    def __init__(self, log_level, message):
        self.logLevel = log_level
        self.message = message

    def get_log_level(self):
        return self.logLevel

    def get_message(self):
        return self.message

    def set_log_level(self, log_level):
        self.logLevel = log_level

    def set_message(self, message):
        self.message = message
    
    def to_dict(self):
        return {
            "level": logging.getLevelName(self.logLevel),
            "message": self.message
        }


# Exceptions
 

In [21]:
class RuleParsingException(Exception):

    def __init__(self, message: str = "Rule parsing error", cause: Exception | None = None):
        super().__init__(message)
        self.cause = cause

    def __str__(self) -> str:
        return f"RuleParsingException: {super().__str__()}"

### SmartHome.py Overview

**Integrating code**
- Acts as the primary controller, integrating all device and data structure functionalities  
- Manages device objects for efficient monitoring and control  
- Enforces power consumption thresholds to optimize energy usage  
- Maintains, checks, and updates battery levels across connected devices  

**Logging**
- Logs key events, enhancing transparency and debugging  

**Priority Management**
- Coordinates with the Priority Queue to handle high-priority device tasks  
- Leverages the Linked List to manage and execute automation rules in a defined order  
- Ensures continuous, coordinated operation among devices, rules, and system events

In [22]:
from concurrent.futures import ThreadPoolExecutor
import random

class SmartHome:

    def __init__(self, threshold: float, ideal_temp: int, simulate: bool):

        self.tick_count: int = 0
        self.threshold: float = threshold
        self.ideal_temp: int = ideal_temp
        self.simulate: bool = simulate
        self.power_consumption: float = 0.0 

        self.lock: threading.Lock = threading.Lock() 
        self.random: random.Random = random.Random()

        self.group_map: dict[str, DeviceGroup] = {}
        self.type_map: dict[str, DeviceType] = {}
        self.location_map: dict[str, DeviceLocation] = {}

        self.powered_on_devices: list[Device] = []
        self.powered_off_devices: list[Device] = []

        self.device_queue: PriorityQueue = PriorityQueue()
        self.power_reducible_devices: PriorityQueue = PriorityQueue()
        self.turn_back_on_devices: PriorityQueue = PriorityQueue()

        self.logging_list: LinkedList = LinkedList()
        self.power_consumption_log_list: LinkedList = LinkedList()
        self.device_battery_log_list: LinkedList = LinkedList()

        self.rule_list: LinkedList = LinkedList()

        self.info_tasks: list[str] = []
        self.warning_tasks: list[str] = []
        self.severe_tasks: list[str] = []
        self.power_consumption_tasks: list[str] = []
        self.device_battery_tasks: list[str] = []

        logging.basicConfig( 
            level=logging.DEBUG, format="%(asctime)s - %(levelname)s - %(message)s"
        )
        self.power_consumption_logger = logging.getLogger("PowerConsumptionLog")
        self.logger = logging.getLogger(__name__) 
        self.device_battery_logger = logging.getLogger("DeviceBatteryLog")

        self.scheduler = None 
        self.tick_thread = None
        self.logging_thread = None
        self.rule_thread = None

        self.tick_running = False
        self.logging_running = False
        self.rule_running = False

        self._initialize_loggers() 
        self._initialize_maps_and_locations() 
        self._initialize_scheduler()

    def _initialize_maps_and_locations(self):
        """Initializes maps for groups, types, and locations."""

        for deviceGroup in DeviceGroupEnum:
            self.group_map[deviceGroup.name] = DeviceGroup(deviceGroup.name)

        for deviceType in DeviceTypeEnum:
            self.type_map[deviceType.name] = DeviceType(deviceType.name)

        for location in DeviceLocationEnum:
            devLocation = DeviceLocation(location.name)
            self.location_map[location.name] = devLocation

            devLocation.set_temperature(self.random.randint(10, 45))

    def _add_to_group_and_type(self, device: Device):

        self.group_map[device.get_device_group().name].add_device(device)
        self.type_map[device.get_device_type().name].add_device(device)
        self.location_map[device.get_location().name].add_device(device)

    def _add_to_power_reducible(self, device: Device):
        """Adds device to device_queue and power_reducible_devices if applicable"""
        try:
            device_type_prio = device.get_device_type().get_priority()[0]
            device_group_prio = device.get_device_group().get_priority()
        except e:
            self.add_log(logging.ERROR, f"Could not get priority for device {device.get_device_name()}: {e}")
            return 

        if device_type_prio == float("inf"): 
            return

        location = self.location_map[device.get_location().name]

        priority = device_type_prio + device_group_prio + (location.get_people() * 10)

        task = Task(device, priority)
        print("ENQUEING TASK")
        self.device_queue.enqueue(task)

        if device.get_power_level() != 0:
            power_reducible_task = Task(device, priority)
            self.power_reducible_devices.enqueue(power_reducible_task)

    def create_device(
            self,
            device_name: str,
            device_type: DeviceTypeEnum,
            device_group: DeviceGroupEnum,
            location: DeviceLocationEnum,
            is_turned_on: bool = False,
            battery_level: float = 100.0,
            power_consumption: float = 0.0,
            max_battery_capacity: int = 0,
            power_level: int = 1,

    ) -> Device:
        """Creates a Device or AirConditioner instance."""

        device_id = len(self.get_devices()) + 1 

        if device_group == DeviceGroupEnum.AIRCONDITIONERS:
             return AirConditioner(
                device_id=device_id,
                device_name=device_name,
                device_type=device_type, 
                location=location,       
                device_group=device_group, 
                is_turned_on=is_turned_on,
                battery_level=battery_level,
                base_power_consumption=power_consumption,
                max_battery_capacity=max_battery_capacity,
                current_battery_capacity=max_battery_capacity if battery_level > 0 else 0, 
                power_level=power_level,
                is_on_battery=battery_level > 0 and max_battery_capacity > 0,
                turned_on_time=int(time.time()) if is_turned_on else 0,
                is_interacted=False,
            )
        else:
            return Device(
                device_id=device_id,
                device_name=device_name,
                device_type=device_type,
                location=location,
                device_group=device_group,
                is_turned_on=is_turned_on,
                battery_level=battery_level,
                base_power_consumption=power_consumption,
                max_battery_capacity=max_battery_capacity,
                current_battery_capacity=max_battery_capacity if battery_level > 0 else 0,
                power_level=power_level,
                is_on_battery=battery_level > 0 and max_battery_capacity > 0,
                turned_on_time=int(time.time()) if is_turned_on else 0,
                is_interacted=False,
            )

    def add_device(self, device: Device):
        """Adds a device to the system and relevant collections."""
        if (device.get_device_group() == DeviceGroupEnum.AIRCONDITIONERS and not isinstance(device, AirConditioner)):
            self.add_log(logging.WARNING, f"Device {device.get_device_name()} added with group AIRCONDITIONERS but is not an AirConditioner instance. Recreating.")

            device = AirConditioner(
                device.get_device_id(), device.get_device_name(), device.get_device_type(),
                device.get_location(), device.get_device_group(), device.is_turned_on(),
                device.get_battery_level(), device.get_base_power_consumption(),
                int(device.get_battery_capacity()), device.get_current_battery_capacity(),
                device.get_power_level(), device.is_on_battery_power(),
                device.get_turned_on_time(), device.get_interaction_state()
            )

        if device.is_turned_on():
            if device not in self.powered_on_devices:
                self.powered_on_devices.append(device)
                device.set_turned_on_time(int(time.time()))

            self._add_to_power_reducible(device)
        else:
            if device not in self.powered_off_devices:
                self.powered_off_devices.append(device)

        self._add_to_group_and_type(device)

        self.logger.info(f"Device added: {device.get_device_name()}")

    def turn_on_device(self, device: Device):

        device.set_turned_on(True)
        device.set_turned_on_time(int(time.time()))

        if device not in self.powered_on_devices:
            self.powered_on_devices.append(device)

        try:
            self.powered_off_devices.remove(device)
        except ValueError:
            print(f"Error turning device {device.get_device_name()} on: {e}")


        self._add_to_power_reducible(device)

        self.add_log(logging.INFO, f"Device turned ON: {device.get_device_name()}")

    def turn_off_device(self, device: Device):
        device.set_turned_on(False)

        if device not in self.powered_off_devices:
            self.powered_off_devices.append(device)

        try:
            self.powered_on_devices.remove(device)
        except e:
            print(f"Error turning device {device.get_device_name()} off: {e}")

        self.device_queue.remove_task(device)
        self.power_reducible_devices.remove_task(device)

        self.add_log(logging.INFO, f"Device turned OFF: {device.get_device_name()}")

    def remove_device(self, device: Device):

        try:
            self.powered_on_devices.remove(device)
        except ValueError:
            pass
        try:
            self.powered_off_devices.remove(device)
        except ValueError:
            pass

        group_name = device.get_device_group().name
        type_name = device.get_device_type().name
        location_name = device.get_location().name

        if group_name in self.group_map:
            self.group_map[group_name].remove_device(device)
        if type_name in self.type_map:
            self.type_map[type_name].remove_device(device)
        if location_name in self.location_map:
            self.location_map[location_name].remove_device(device)

        self.device_queue.remove_task(device)
        self.power_reducible_devices.remove_task(device)

        self.logger.info(f"Device removed: {device.get_device_name()}")
        return True

    def get_device_by_name(self, name: str) -> Device | None:
        """Finds a device by name (case-insensitive)."""
        lname = name.lower()
        for device in list(self.powered_on_devices):
            if device.get_device_name().lower() == lname:
                return device
        for device in list(self.powered_off_devices):
            if device.get_device_name().lower() == lname:
                return device

        return None

    def get_device_by_id(self, device_id: int) -> Device | None:
        """Finds a device by its unique ID."""

        for device in list(self.powered_on_devices):
            if device.get_device_id() == device_id:
                return device
        for device in list(self.powered_off_devices):
            if device.get_device_id() == device_id:
                return device

        return None

    def turn_off_devices_by_group(self, group_name: str):
        if group_name not in self.group_map:
            self.logger.warning(f"Attempted to turn off devices for non-existent group: {group_name}")
            return

        devices_to_turn_off = []

        devices_to_turn_off = self.group_map[group_name].get_devices()

        for device in devices_to_turn_off:
            self.turn_off_device(device) 

    def turn_on_devices_by_group(self, group_name: str):
        if group_name not in self.group_map:
            self.add_log(logging.ERROR, f"Attempted to turn on devices for non-existent group: {group_name}")
            print(f"Attempted to turn on devices for non-existent group: {group_name}")
            return
        
        devices_to_turn_on = self.group_map[group_name].get_devices()
        for device in devices_to_turn_on:
            self.turn_on_device(device)

    def turn_off_devices_by_type(self, type_name: str):
        if type_name not in self.type_map:
            self.add_log(logging.ERROR, f"Attempted to turn off devices for non-existent type: {type_name}")
            print(f"Attempted to turn off devices for non-existent type: {type_name}")
            return
         
        devices_to_turn_off = self.type_map[type_name].get_devices()
        for device in devices_to_turn_off:
            self.turn_off_device(device)

    def turn_on_devices_by_type(self, type_name: str):
        if type_name not in self.type_map:
            self.logger.warning(f"Attempted to turn on devices for non-existent type: {type_name}")
            print(f"Attempted to turn on devices for non-existent type: {type_name}")
            return
        
        devices_to_turn_on = self.type_map[type_name].get_devices()
        for device in devices_to_turn_on:
            self.turn_on_device(device)

    def turn_off_devices_by_location(self, location_name: str):
        if location_name not in self.location_map:
            self.logger.warning(f"Attempted to turn off devices for non-existent location: {location_name}")
            print(f"Attempted to turn off devices for non-existent location: {location_name}")
            return
        
        devices_to_turn_off = self.location_map[location_name].get_devices()
        for device in devices_to_turn_off:
            self.turn_off_device(device)

    def turn_on_devices_by_location(self, location_name: str):
        if location_name not in self.location_map:
            self.logger.warning(f"Attempted to turn on devices for non-existent location: {location_name}")
            print(f"Attempted to turn on devices for non-existent location: {location_name}")
            return
        
        devices_to_turn_on = self.location_map[location_name].get_devices()
        for device in devices_to_turn_on:
            self.turn_on_device(device)

    def turn_off_all_devices(self):
        devices_to_turn_off = list(self.powered_on_devices)
        for device in devices_to_turn_off:
            self.turn_off_device(device)

    def turn_on_all_devices(self):
        devices_to_turn_on = list(self.powered_off_devices)
        for device in devices_to_turn_on:
            self.turn_on_device(device)

    def get_device(self, identifier: str | int) -> Device | None:
        """Gets a device by ID (int) or name (str)."""
        if isinstance(identifier, int):
            return self.get_device_by_id(identifier)
        elif isinstance(identifier, str):
            return self.get_device_by_name(identifier)
        else:
            self.add_log(logging.ERROR, f"Invalid identifier type for get_device: {type(identifier)}")
            print(f"Invalid identifier type for get_device: {type(identifier)}")
            return None

    def add_person(self, location_enum: DeviceLocationEnum):
        """Adds a person to a location and updates device priorities."""

        if isinstance(location_enum, str):
            location_name = location_enum
        else: location_name = location_enum.name
        
        if location_name not in self.location_map:
            self.add_log(logging.ERROR, f"Attempted to add person to non-existent location: {location_name}")
            print(f"Attempted to add person to non-existent location: {location_name}")
            return

        location_obj = self.location_map[location_name]
        location_obj.add_people(1) 

        devices_in_location: list[Device] = location_obj.get_devices()
        for device in devices_in_location:
            if device.is_turned_on():

                task = self.device_queue.get_task(device)
                if task:
                    new_priority = task.priority + 10
                    updated = self.device_queue.update_priority(task.task, new_priority)
                    if not updated:
                        self.add_log(logging.ERROR, f"Failed to update priority for {device.get_device_name()} in device_queue")
                        print(f"Failed to update priority for {device.get_device_name()} in device_queue")

                power_task = self.power_reducible_devices.get_task(device)
                if power_task:
                    new_priority = power_task.priority + 10
                    updated = self.power_reducible_devices.update_priority(power_task.task, new_priority)
                    if not updated:
                        self.add_log(logging.ERROR, f"Failed to update priority for {device.get_device_name()} in power_reducible_devices")
                        print(f"Failed to update priority for {device.get_device_name()} in power_reducible_devices")

        self.logger.info(f"Person added to {location_name}. People: {location_obj.get_people()}")

    def remove_person(self, location_enum: DeviceLocationEnum):
        """Removes a person from a location and updates device priorities."""
        if isinstance(location_enum, str):
            location_name = location_enum

        else: location_name = location_enum.name
        if location_name not in self.location_map:
            self.logger.warning(f"Attempted to remove person from non-existent location: {location_name}")
            return

        location_obj = self.location_map[location_name]
        if location_obj.get_people() == 0:
             return 

        location_obj.remove_people(1) 

        devices_in_location: list[Device] = location_obj.get_devices()
        for device in devices_in_location:
             if device.is_turned_on():

                task = self.device_queue.get_task(device)
                if task:
                    new_priority = task.priority - 10
                    updated = self.device_queue.update_priority(task.task, new_priority)
                    if not updated:
                        self.add_log(logging.ERROR, f"Failed to update priority for {device.get_device_name()} in device_queue")

                power_task: Task = self.power_reducible_devices.get_task(device)
                if power_task:
                    new_priority = power_task.priority - 10
                    updated = self.power_reducible_devices.update_priority(power_task.task, new_priority)
                    if not updated:
                        self.add_log(logging.ERROR, f"Failed to update priority for {device.get_device_name()} in power_reducible_devices")

        self.logger.info(f"Person removed from {location_name}. People: {location_obj.get_people()}")

    def _initialize_scheduler(self):

        self.scheduler = ThreadPoolExecutor(max_workers=3, thread_name_prefix="SmartHomeWorker")

        try:
            self.start_tick()
            self.start_logging()
            self.start_rule_execution()
            self.add_log(logging.INFO, "Scheduler initialized and background tasks started.")
        except Exception as e:
            print(e)


    def start_tick(self):

        if not self.tick_running:
            self.tick_running = True
            self.tick_thread = self.scheduler.submit(self._run_tick_periodically)
            self.add_log(logging.INFO,"Tick thread started.")

    def _run_tick_periodically(self):
        """Worker function for the tick thread."""
        while self.tick_running: 
            start_time = time.monotonic() 
            try:
                if not self.tick_running:
                    break
                self.tick() 

            except Exception as e:
                print(e)
                self.add_log(logging.ERROR, f"Error during tick execution: {e}")

            end_time = time.monotonic()
            execution_time = end_time - start_time
            sleep_time = max(0.01, 1.0 - execution_time) 

            if not self.tick_running:
                break

            time.sleep(sleep_time)
        self.add_log(logging.INFO, "Tick thread finished.")

    def stop_tick(self):
        if self.tick_running:
            self.tick_running = False
            self.add_log(logging.INFO, "Tick thread stop requested.")

    def tick(self):
        """Performs a single tick of the simulation/update logic."""

        try:
            self.tick_task()
        except Exception as e:
            print(e)
            self.add_log(logging.ERROR, f"Exception during tick task: {e}")

    def check_power_consumption(self):
    
        curr_power_consumption = self.calculate_current_power_consumption_internal()
        
        if curr_power_consumption > self.threshold:
            reduce_power_task: Task = self.power_reducible_devices.dequeue()
            if reduce_power_task is None:
                self.add_log(logging.INFO, "No devices to reduce power consumption")
                print("No devices to reduce power consumption")
                return
                
            device: Device = reduce_power_task.task
            
            if curr_power_consumption - (device.get_base_power_consumption() * (device.get_power_level() - 1)) > self.threshold:
                self.power_reducible_devices.enqueue(reduce_power_task)
                remove_task: Task = self.device_queue.dequeue()
                if remove_task is None:
                    self.add_log(log_level=logging.ERROR, message="No devices to turn off")
                    print("No devices to turn off")
                    return
                    
                remove_device: Device = remove_task.task
                
                self.add_power_log(logging.INFO, f"Reducing power consumption by turning off {device.get_device_name()}")
                
                self.add_rule(self.parse_rule(f"turn {remove_device.get_device_id()} off"))
                self.turn_back_on_devices.enqueue(Task(device, -remove_task.priority))
            else:
                self.add_rule(self.parse_rule(f"set {device.get_device_id()} 1"))
        else:
            turn_back_on_task: Task = self.turn_back_on_devices.dequeue()
            if turn_back_on_task is not None:
                turn_back_on_device: Device = turn_back_on_task.task
                
                if curr_power_consumption + (turn_back_on_device.get_base_power_consumption() * turn_back_on_device.get_power_level()) > self.threshold:
                    self.add_log(logging.INFO, f"Not turning back on {turn_back_on_task.task.get_device_name()}")
                    turn_back_on_task.priority += 3
                    self.turn_back_on_devices.enqueue(turn_back_on_task)
                    return
                    
                self.add_log(logging.INFO, f"Turning back on {turn_back_on_task.task.get_device_name()}")
                self.add_rule(self.parse_rule(f"turn {turn_back_on_task.task.get_device_id()} on"))

    def tick_task(self):
        """The core logic executed in each tick."""

        self.tick_count += 1
        current_tick = self.tick_count

        if current_tick % 2 == 0: 
            if self.simulate:
                try:
                    self.simulate_device_change() 
                except Exception as e:
                    print(e)
                    self.add_log(logging.ERROR, f"Error during simulate_device_change: {e}")
            try:
                self.realistic_power_consumption() 
            except Exception as e:
                print(e)
                self.add_log(logging.ERROR, f"Error during realistic_power_consumption: {e}")

        try:
            self.log_power_consumption() 
        except Exception as e:
            print(e)
            self.add_log(logging.ERROR, f"Error during log_power_consumption: {e}")
        try:
            self.reduce_battery_tick() 
        except Exception as e:
            print(e)
            self.add_log(logging.ERROR, f"Error during reduce_battery_tick: {e}")
        try:
            self.check_each_device() 
        except Exception as e:
            print(e)
            self.add_log(logging.ERROR, f"Error during check_each_device: {e}")
        try:
            self.check_each_location() 
        except Exception as e:
            print(e)
            self.add_log(logging.ERROR, f"Error during check_each_location: {e}")
        try:
            self.check_power_consumption() 
        except Exception as e:
            print(e)
            self.add_log(logging.ERROR, f"Error during check_power_consumption: {e}")

    def start_rule_execution(self):
        if not self.rule_running:
            self.rule_running = True
            self.rule_thread = self.scheduler.submit(self._run_execute_rules_periodically)
            self.logger.info("Rule execution thread started.")

    def _run_execute_rules_periodically(self):
        """Worker function for executing rules."""
        while self.rule_running:
            start_time = time.monotonic()
            try:
                if not self.rule_running:
                    break

                self.execute_rules() 

            except Exception as e:
                self.logger.error(f"Error during rule execution: {e}", exc_info=True)

            end_time = time.monotonic()
            execution_time = end_time - start_time

            sleep_time = max(0.01, 1.0 - execution_time)

            if not self.rule_running:
                break
            time.sleep(sleep_time)
        self.logger.info("Rule execution thread finished.")

    def stop_rule_execution(self):
        if self.rule_running:
            self.rule_running = False
            self.logger.info("Rule execution thread stop requested.")

    def execute_rules(self):
        """Executes rules from the rule list."""

        rule_to_execute = self.rule_list.remove_beginning()

        while rule_to_execute is not None:
            try:
                self.execute_rule(rule_to_execute) 
            except RuleParsingException as rpe:
                self.add_log(logging.ERROR,f"Rule Execution Error (Rule: {rule_to_execute}): {rpe}")
            except Exception as e:
                self.add_log(logging.ERROR,f"Unexpected Error executing rule (Rule: {rule_to_execute}): {e}")

            rule_to_execute = self.rule_list.remove_beginning()

    def execute_rule(self, rule: Rule):
        if rule.get_device_id() != -1:
            print(f"Processing device with ID: {rule.get_device_id()}")
            device = self.get_device(rule.get_device_id())
            if device is None:
                print(f"ERROR: Device with ID {rule.get_device_id()} not found")
                raise RuleParsingException(f"Device with ID {rule.get_device_id()} not found for rule.")

            if rule.get_flip_state():
                print(f"Executing flip_state for device: {device.get_device_name()}")
                self.flip_device_state(device)

            elif rule.get_set_power_level():
                print(f"Executing set_power_level for device: {device.get_device_name()}")
                if not (0 <= rule.get_power_level() <= 5):
                    print(f"ERROR: Invalid power level {rule.get_power_level()} for device {device.get_device_name()}")
                    raise RuleParsingException(f"Invalid power level {rule.get_power_level()} for device {device.get_device_name()}. Must be 0-5.")

                print(f"Setting power level to: {rule.get_power_level()}")
                with self.lock:
                    device.set_power_level(rule.get_power_level())

                self.logger.info(f"Rule: Set power level of {device.get_device_name()} to {rule.get_power_level()}")
            elif rule.get_turn_on():
                print(f"Executing turn_on for device: {device.get_device_name()}")
                self.turn_on_device(device)
            elif rule.get_turn_off():
                print(f"Executing turn_off for device: {device.get_device_name()}")
                self.turn_off_device(device)

        if rule.get_group_name():
            print(f"Processing group: {rule.get_group_name()}")
            group = self.get_group(rule.get_group_name())
            if group is None:
                print(f"ERROR: Group '{rule.get_group_name()}' not found")
                raise RuleParsingException(f"Group '{rule.get_group_name()}' not found for rule.")

            if rule.get_turn_group_on():
                print(f"Executing turn_on for group: {group.get_group_name()}")
                self.turn_on_devices_by_group(group.get_group_name())
            elif rule.get_turn_group_off():
                print(f"Executing turn_off for group: {group.get_group_name()}")
                self.turn_off_devices_by_group(group.get_group_name())

        if rule.get_type_name():
            print(f"Processing type: {rule.get_type_name()}")
            type_ = self.get_type(rule.get_type_name())
            if type_ is None:
                print(f"ERROR: Type '{rule.get_type_name()}' not found")
                raise RuleParsingException(f"Type '{rule.get_type_name()}' not found for rule.")

            if rule.get_turn_type_on():
                print(f"Executing turn_on for type: {rule.get_type_name()}")
                self.turn_on_devices_by_type(rule.get_type_name())
            elif rule.get_turn_type_off():
                print(f"Executing turn_off for type: {rule.get_type_name()}")
                self.turn_off_devices_by_type(rule.get_type_name())

        if rule.get_location_name():
            print(f"Processing location: {rule.get_location_name()}")

            location = self.get_location(rule.get_location_name())
            if location is None:
                print(f"ERROR: Location '{rule.get_location_name()}' not found")
                raise RuleParsingException(f"Location '{rule.get_location_name()}' not found for rule.")

            if rule.get_turn_location_on():
                print(f"Executing turn_on for location: {rule.get_location_name()}")
                self.turn_on_devices_by_location(rule.get_location_name())

            elif rule.get_turn_location_off():
                print(f"Executing turn_off for location: {rule.get_location_name()}")
                self.turn_off_devices_by_location(rule.get_location_name())


    def flip_device_state(self, device: Device):
        is_currently_on = device.is_turned_on()

        if is_currently_on:
            self.turn_off_device(device)
        else:
            self.turn_on_device(device)

    def parse_rule(self, rule_string: str) -> Rule | None:
        """Parses a string command into a Rule object."""
        tokens = rule_string.lower().split()
        if not tokens:
            raise RuleParsingException("Rule string is empty.")

        command = tokens[0]
        args = tokens[1:]

        if command == "flip":
            self.check_token_size(args, 1) 
            device = self.check_token_for_device(args[0])
            return Rule(device_id=device.get_device_id(), flip_state=True)

        elif command == "turn":
            self.check_token_size(args, 2) 
            device = self.check_token_for_device(args[0])
            state = self.check_token_on_off(args[1])
            return Rule(device_id=device.get_device_id(), turn_on=state, turn_off=not state)

        elif command == "set":
            self.check_token_size(args, 2) 
            device = self.check_token_for_device(args[0])
            if not self.is_numeric(args[1]):
                    raise RuleParsingException(f"Invalid power level '{args[1]}': Must be numeric.")
            power_level = int(args[1])
            if not (0 <= power_level <= 5):
                    raise RuleParsingException(f"Invalid power level {power_level}: Must be between 0 and 5.")
            return Rule(device_id=device.get_device_id(), set_power_level=True, power_level=power_level)

        elif command == "group":
            self.check_token_size(args, 2) 
            group_name = args[0].upper() 
            if group_name not in self.group_map:
                    raise RuleParsingException(f"Group '{group_name}' not found.")
            state = self.check_token_on_off(args[1])
            return Rule(group_name=group_name, turn_group_on=state, turn_group_off=not state)

        elif command == "type":
            self.check_token_size(args, 2) 
            type_name = args[0].upper()
            if type_name not in self.type_map:
                    raise RuleParsingException(f"Type '{type_name}' not found.")
            state = self.check_token_on_off(args[1])
            return Rule(type_name=type_name, turn_type_on=state, turn_type_off=not state)

        elif command == "location":
                self.check_token_size(args, 2) 
                location_name = args[0].upper()
                if location_name not in self.location_map:
                    raise RuleParsingException(f"Location '{location_name}' not found.")
                state = self.check_token_on_off(args[1])
                return Rule(location_name=location_name, turn_location_on=state, turn_location_off=not state)

        else:
            valid_commands = ["flip", "turn", "set", "group", "type", "location"] 
            raise RuleParsingException(f"Invalid rule command '{command}'. Valid commands: {valid_commands}")

    def add_rule(self, rule: Rule):
        """Adds a rule to the end of the execution queue."""
        if rule: 
            self.rule_list.add_end(rule)

    def add_immediate_rule(self, rule: Rule):
        """Adds a rule to the front of the execution queue."""
        if rule:
            self.rule_list.add_front(rule)

    def _initialize_loggers(self):
        """Sets up file handlers and formatters for logging."""

        try:

            self.logger.propagate = False
            self.power_consumption_logger.propagate = False
            self.device_battery_logger.propagate = False

            self.logger.setLevel(logging.DEBUG) 
            self.power_consumption_logger.setLevel(logging.INFO)
            self.device_battery_logger.setLevel(logging.INFO)

            formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
            power_formatter = logging.Formatter("%(asctime)s - %(message)s") 

            info_file_handler = logging.FileHandler("Info.log", mode='w') 
            info_file_handler.setLevel(logging.INFO)
            info_file_handler.setFormatter(formatter)
            info_file_handler.addFilter(lambda record: record.levelno == logging.INFO) 
            self.logger.addHandler(info_file_handler)

            warning_file_handler = logging.FileHandler("Warning.log", mode='w')
            warning_file_handler.setLevel(logging.WARNING)
            warning_file_handler.setFormatter(formatter)
            warning_file_handler.addFilter(lambda record: record.levelno == logging.WARNING)
            self.logger.addHandler(warning_file_handler)

            severe_file_handler = logging.FileHandler("Error.log", mode='w')
            severe_file_handler.setLevel(logging.ERROR) 
            severe_file_handler.setFormatter(formatter)

            self.logger.addHandler(severe_file_handler)

            power_consumption_file_handler = logging.FileHandler("PowerConsumption.log", mode='w')
            power_consumption_file_handler.setLevel(logging.INFO)
            power_consumption_file_handler.setFormatter(power_formatter)
            self.power_consumption_logger.addHandler(power_consumption_file_handler)

            device_battery_file_handler = logging.FileHandler("DeviceBattery.log", mode='w')
            device_battery_file_handler.setLevel(logging.INFO)
            device_battery_file_handler.setFormatter(power_formatter)
            self.device_battery_logger.addHandler(device_battery_file_handler)

            self.logger.info("Loggers initialized successfully.")

        except Exception as e:

            print(f"FATAL: Failed to initialize loggers: {e}")

    def start_logging(self):
        if not self.logging_running:
            self.logging_running = True
            self.logging_thread = self.scheduler.submit(self._run_log_periodically)

    def _run_log_periodically(self):
        """Worker function for writing logs from queues."""

        while self.logging_running:
            start_time = time.monotonic()
            try:
                if not self.logging_running:
                    break

                self.log() 

            except Exception as e:

                 print(f"Error in logging thread: {e}")

            end_time = time.monotonic()
            execution_time = end_time - start_time

            sleep_time = max(0.01, 2.0 - execution_time)

            if not self.logging_running:
                break
            time.sleep(sleep_time)

        print("Logging thread finished.")

    def stop_logging(self):
        if self.logging_running:
            self.logging_running = False

            print("Logging thread stop requested.")

    def log(self):
        """Processes log tasks from the internal queues."""

        task: LogTask = self.logging_list.remove_beginning()
        while task is not None:
            self.logger.log(task.logLevel, task.message)
            task = self.logging_list.remove_beginning()

        power_task: LogTask = self.power_consumption_log_list.remove_beginning()
        while power_task is not None:
            self.power_consumption_logger.log(power_task.logLevel, power_task.message)
            power_task = self.power_consumption_log_list.remove_beginning()

        battery_task: LogTask = self.device_battery_log_list.remove_beginning()
        while battery_task is not None:
            self.device_battery_logger.log(battery_task.logLevel, battery_task.message)
            battery_task = self.device_battery_log_list.remove_beginning()

    def add_log(self, log_level, message):
        """Adds a message to the general logging queue and UI task list."""

        log_task = LogTask(log_level, message)
        self.logging_list.add_end(log_task)

        if log_level == logging.INFO:
            self.info_tasks.append(message)
        elif log_level == logging.WARNING:
            self.warning_tasks.append(message)
        elif log_level >= logging.ERROR:
            self.severe_tasks.append(message)

    def add_power_log(self, log_level, message):

        log_task = LogTask(log_level, message)
        self.power_consumption_log_list.add_end(log_task)
        self.power_consumption_tasks.append(message)

        if log_level != logging.INFO:
            self.logging_list.add_end(log_task)

            if log_level == logging.WARNING:
                self.warning_tasks.append(message)
            elif log_level >= logging.ERROR:
                self.severe_tasks.append(message)

    def add_battery_log(self, log_level, message):

        log_task = LogTask(log_level, message)
        self.device_battery_log_list.add_end(log_task)
        self.device_battery_tasks.append(message)

        if log_level != logging.INFO:
            self.logging_list.add_end(log_task)

            if log_level == logging.WARNING:
                self.warning_tasks.append(message)
            elif log_level >= logging.ERROR:
                self.severe_tasks.append(message)

    def calculate_current_base_power_consumption(self) -> float:
        """Calculates total base power consumption of ON devices."""
        total_base_power = sum(device.get_base_power_consumption() for device in list(self.powered_on_devices))

        return total_base_power

    def calculate_current_power_consumption(self) -> float:
        """Calculates total current power consumption (base * level) of ON devices."""
        total_power = self.calculate_current_power_consumption_internal()

        return total_power

    def calculate_current_power_consumption_internal(self) -> float:
        return sum(
                device.get_base_power_consumption() * max(1, device.get_power_level()) 
                for device in self.powered_on_devices 
            )

    def log_power_consumption(self):
        """Calculates and logs the current power consumption state."""

        self.power_consumption = self.calculate_current_power_consumption_internal()
        current_consumption = self.power_consumption
        threshold = self.threshold

        if current_consumption > threshold * 1.5:
            level = logging.CRITICAL
            percent_above = (current_consumption - threshold) / threshold * 100
            message = f"Power consumption is {current_consumption:.2f}W, which is {percent_above:.2f}% above the threshold ({threshold}W)"
        elif current_consumption > threshold:
            level = logging.WARNING
            percent_above = (current_consumption - threshold) / threshold * 100
            message = f"Power consumption is {current_consumption:.2f}W, which is {percent_above:.2f}% above the threshold ({threshold}W)"
        else:
            level = logging.INFO
            percent_of = (current_consumption / threshold * 100) if threshold > 0 else 0
            message = f"Power consumption - {current_consumption:.2f}W, {percent_of:.2f}% of threshold ({threshold}W)"

            self._add_power_log_internal(level, message)

    def _add_power_log_internal(self, log_level, message):
        """Internal version of add_power_log assuming lock is held."""

        log_task = LogTask(log_level, message)
        self.power_consumption_log_list.add_end(log_task)
        self.power_consumption_tasks.append(message)
        if log_level != logging.INFO:
            self.logging_list.add_end(log_task)
            if log_level == logging.WARNING:
                self.warning_tasks.append(message)
            elif log_level >= logging.ERROR:
                self.severe_tasks.append(message)

    def get_power_consumption(self) -> float:
        """Returns the last calculated power consumption."""
        return self.power_consumption

    def realistic_power_consumption(self):
        """Simulates random fluctuations in device base power consumption."""
        for device in self.powered_on_devices:

            rand_val = self.random.random()
            current_base = device.get_base_power_consumption()
            if rand_val >= 0.95:
                increase = current_base * self.random.uniform(0.1, 0.3)
                device.set_base_power_consumption(current_base + increase)
            elif rand_val <= 0.05:
                decrease = current_base * self.random.uniform(0.1, 0.3)
                device.set_base_power_consumption(max(0.1, current_base - decrease))

    def reduce_battery_tick(self):
        """Reduces battery for active devices on battery power."""
        devices_to_turn_off = []

        for device in list(self.powered_on_devices):

            if device.is_on_battery_power() and device.get_battery_capacity() > 0:

                current_level_before = int(device.get_battery_level())
                self.reduce_battery_level_internal(device)
                current_level_after = int(device.get_battery_level())

                level_percent = device.get_battery_level()
                if current_level_after != current_level_before:
                    self._add_battery_log_internal(
                        logging.INFO, f"Battery level of {device.get_device_name()} is now {level_percent:.0f}%"
                    )

                if level_percent < 10:
                    self._add_battery_log_internal(
                        logging.CRITICAL, f"CRITICAL: Battery level of {device.get_device_name()} is below 10%!"
                    )
                elif level_percent < 20:
                    self._add_battery_log_internal(
                        logging.WARNING, f"WARNING: Battery level of {device.get_device_name()} is below 20%!"
                    )

                if device.get_current_battery_capacity() <= 0:
                    self._add_battery_log_internal(
                        logging.ERROR, f"ERROR: Battery of {device.get_device_name()} has run out! Turning off."
                    )

                    devices_to_turn_off.append(device)

        for device in devices_to_turn_off:
            self.turn_off_device(device) 

    def reduce_battery_level_internal(self, device: Device):
        """Internal helper to reduce battery level (assumes lock held)."""

        if not device.is_on_battery_power() or device.get_battery_capacity() <= 0:
            return

        consumption_this_tick = device.get_base_power_consumption() * max(1, device.get_power_level())
        new_capacity = device.get_current_battery_capacity() - consumption_this_tick
        device.set_current_battery_capacity(max(0, new_capacity)) 

        new_level = (device.get_current_battery_capacity() / device.get_battery_capacity()) * 100
        device.set_battery_level(new_level)

    def _add_battery_log_internal(self, log_level, message):
        """Internal version of add_battery_log assuming lock is held."""

        log_task = LogTask(log_level, message)
        self.device_battery_log_list.add_end(log_task)
        self.device_battery_tasks.append(message)
        if log_level != logging.INFO:
            self.logging_list.add_end(log_task)
            if log_level == logging.WARNING:
                self.warning_tasks.append(message)
            elif log_level >= logging.ERROR:
                self.severe_tasks.append(message)

    def add_location(self, location_name: str):
        """Adds a new location if it doesn't exist."""
        lname = location_name.upper() 

        if lname in self.location_map:
            self.add_log(logging.WARNING, f"Location '{lname}' already exists, add attempt ignored.")
          
        else:
            new_location = DeviceLocation(lname)
            new_location.set_temperature(self.random.randint(10, 45))
            self.location_map[lname] = new_location

            self.add_log(logging.INFO, f"Location '{lname}' added.")

    def check_each_location(self):
        """Checks temperature and other conditions in each location."""
        locations = list(self.location_map.values())

        for location in locations:
            temp = location.get_temperature() 

            if temp > 40:
                self.add_log(logging.CRITICAL, f"CRITICAL: Temperature in {location.location} is {temp}")
            elif temp > 35:
                self.add_log(logging.WARNING, f"WARNING: Temperature in {location.location} is {temp}")
            elif temp < 10:
                self.add_log(logging.CRITICAL, f"CRITICAL: Temperature in {location.location} is {temp}")
            elif temp < 15:
                self.add_log(logging.WARNING, f"WARNING: Temperature in {location.location} is {temp}")

            air_conditioner = location.get_device_by_name("AirConditioner") 

            if air_conditioner and isinstance(air_conditioner, AirConditioner):

                is_ac_on = air_conditioner.is_turned_on()
                if is_ac_on:
                    try:
                        self.temp_check(air_conditioner, location)
                    except e:
                        self.add_log(logging.ERROR, f"Error during temp_check for AC in {location}: {e}")

    def temp_check(self, air_conditioner: AirConditioner, location: DeviceLocation):
        """Controls an AirConditioner based on ideal temperature. Adds rules."""
        
        if air_conditioner.get_minutes_since_temp_change() >= 1:
            air_conditioner.set_simulation_temp_change_time(time.time())
            if air_conditioner.get_mode():
                location.set_temperature(location.get_temperature() - air_conditioner.get_power_level())
            else:
                location.set_temperature(location.get_temperature() + air_conditioner.get_power_level())
        
        if self.ideal_temp == location.get_temperature():
            self.add_log(logging.INFO, f"Temperature is ideal, turning off AirConditioner in {location.location}")
            self.add_rule(self.parse_rule(f"set {air_conditioner.get_device_id()} 1"))
            self.add_rule(self.parse_rule(f"turn {air_conditioner.get_device_id()} off"))
            return
        
        if self.ideal_temp - location.get_temperature() > 0:
            air_conditioner.set_mode(False)
            power_level = min(5, int(self.ideal_temp - location.get_temperature()))
            self.add_rule(self.parse_rule(f"set {air_conditioner.get_device_id()} {power_level}"))
        else:
            air_conditioner.set_mode(True)
            power_level = min(5, int(location.get_temperature() - self.ideal_temp))
            self.add_rule(self.parse_rule(f"set {air_conditioner.get_device_id()} {power_level}"))

    def check_each_device(self):
        """Performs checks on individual devices (e.g., high power consumption)."""

        devices_to_check = list(self.powered_on_devices)

        for device in devices_to_check:
            base_power = device.get_base_power_consumption() 
            if base_power > 100: 
                self.add_log(logging.CRITICAL, f"CRITICAL: Base power of {device.get_device_name()} is {base_power:.1f}W (> 100W)!")
            elif base_power > 50:
                self.add_log(logging.WARNING, f"WARNING: Base power of {device.get_device_name()} is {base_power:.1f}W (> 50W)!")

            if isinstance(device,AirConditioner):
                self.temp_check(device, self.location_map[device.get_location().name])
                

    def simulate_device_change(self):
        """Simulates random device state changes (on/off, power level)."""
        devices_to_turn_off: list[Device] = []
        devices_to_turn_on: list[Device] = []
        devices_to_update_level: list[Device] = []

        powered_on_copy = list(self.powered_on_devices)
        powered_off_copy = list(self.powered_off_devices)

        for device in powered_on_copy:

            if not isinstance(device, Device) or isinstance(device, AirConditioner): 
                continue

            random_double = self.random.random()
            if random_double >= 0.8: 
                devices_to_turn_off.append(device)

            elif device.get_power_level() != 0 and random_double <= 0.2: 
                new_level = self.random.randint(1, 5) 
                if new_level != device.get_power_level():
                    devices_to_update_level.append((device, new_level))

        for device in powered_off_copy:
            if not isinstance(device, Device) or isinstance(device, AirConditioner):
                continue

            if self.random.random() >= 0.8: 
                devices_to_turn_on.append(device)

        for device in devices_to_turn_off:
            self.logger.debug(f"Simulating: Turning OFF {device.get_device_name()}")
            self.turn_off_device(device)

        for device, level in devices_to_update_level:

            self.logger.debug(f"Simulating: Setting power level of {device.get_device_name()} to {level}")
            device.set_power_level(level)

        for device in devices_to_turn_on:
            self.logger.debug(f"Simulating: Turning ON {device.get_device_name()}")
            self.turn_on_device(device)

    def is_numeric(self, string: str) -> bool:
        """Checks if a string can be converted to an integer."""
        if isinstance(string, (int, float)): 
            return True
        return string.isdigit() or (string.startswith('-') and string[1:].isdigit())

    def get_devices_by_group(self, group_name: str) -> list[Device]:
        """Gets a list of devices belonging to a specific group."""
        group = self.group_map.get(group_name.upper())
        if group:

             return group.get_devices()
        else:
             return []

    def get_device_groups(self) -> dict[str, DeviceGroup]:
        return dict(self.group_map)

    def get_device_types(self) -> dict[str, DeviceType]:
        return dict(self.type_map)

    def get_device_locations(self) -> dict[str, DeviceLocation]:
        return dict(self.location_map)

    def get_devices_by_type(self, type_name: str) -> list[Device]:
        type_ = self.type_map.get(type_name.upper())
        return type_.get_devices() if type_ else []

    def get_devices_by_location(self, location_name: str) -> list[Device]:
        location = self.location_map.get(location_name.upper())
        return location.get_devices() if location else []

    def check_token_for_device(self, token: str) -> Device:
        """Helper for rule parsing: Finds device by ID or name."""

        device = None
        if self.is_numeric(token):
            device = self.get_device_by_id(int(token)) 
        else:

            device = self.get_device_by_name(token) 

        if device is None:
            raise RuleParsingException(f"Device '{token}' not found.")
        return device

    def check_token_size(self, tokens: list[str], expected_size: int):
        if len(tokens) != expected_size:
            raise RuleParsingException(f"Invalid number of arguments. Expected {expected_size}, got {len(tokens)}: {tokens}")

    def check_token_on_off(self, token: str) -> bool:
        if token == "on":
            return True
        if token == "off":
            return False
        raise RuleParsingException(f"Invalid state '{token}'. Expected 'on' or 'off'.")

    def set_threshold(self, threshold: float):
        self.threshold = threshold

    def set_ideal_temp(self, ideal_temp: int):
        self.ideal_temp = ideal_temp

    def set_simulate(self, simulate: bool):
        self.simulate = simulate

    def get_threshold(self) -> float:
        return self.threshold

    def get_ideal_temp(self) -> int:
        return self.ideal_temp

    def is_simulate(self) -> bool:
        return self.simulate

    def get_powered_on_devices(self) -> list[Device]:
        return list(self.powered_on_devices)

    def get_powered_off_devices(self) -> list[Device]:
        return list(self.powered_off_devices)

    def get_devices(self) -> list[Device]:
        devices = list(self.powered_on_devices)
        devices.extend(list(self.powered_off_devices))
        return devices

    def get_location(self, location: str) -> DeviceLocation | None:
        return self.location_map.get(location.upper())

    def get_group(self, group: str) -> DeviceGroup | None:
        return self.group_map.get(group.upper())

    def get_type(self, type_name: str) -> DeviceType | None:
        return self.type_map.get(type_name.upper())

    def get_power_consumption_tasks(self) -> list[str]:
        return list(self.power_consumption_tasks)
    def get_device_battery_tasks(self) -> list[str]:
        return list(self.device_battery_tasks)
    def get_info_tasks(self) -> list[str]:
        return list(self.info_tasks)
    def get_warning_tasks(self) -> list[str]:
        return list(self.warning_tasks)
    def get_severe_tasks(self) -> list[str]:
        return list(self.severe_tasks)

    def clear_info_tasks(self):
        self.info_tasks.clear()
    def clear_warning_tasks(self):
        self.warning_tasks.clear()
    def clear_severe_tasks(self):
        self.severe_tasks.clear()
    def clear_power_consumption_tasks(self):
        self.power_consumption_tasks.clear()
    def clear_device_battery_tasks(self):
        self.device_battery_tasks.clear()

    def get_device_battery_log_list(self) -> LinkedList:

        return self.device_battery_log_list
    def get_power_consumption_log_list(self) -> LinkedList:
        return self.power_consumption_log_list
    def get_logging_list(self) -> LinkedList:
        return self.logging_list
    def get_rule_list(self) -> LinkedList:
        return self.rule_list

    def get_device_queue(self) -> PriorityQueue:
        return self.device_queue
    def get_power_reducible_devices(self) -> PriorityQueue:
        return self.power_reducible_devices
    def get_turn_back_on_devices(self) -> PriorityQueue:
        return self.turn_back_on_devices

    def shutdown(self):
        """Gracefully shuts down the scheduler and background threads."""
        self.add_log(logging.INFO, "Shutdown requested...")

        self.stop_tick()
        self.stop_rule_execution()
        self.stop_logging() 

        if self.scheduler:
            self.logger.info("Shutting down scheduler...")

            self.scheduler.shutdown(wait=True)
            self.logger.info("Scheduler shut down.")

        self.add_log(logging.INFO,"Closing logger handlers.")
        for handler in self.logger.handlers:
            handler.close()
            self.logger.removeHandler(handler)
        for handler in self.power_consumption_logger.handlers:
            handler.close()
            self.power_consumption_logger.removeHandler(handler)
        for handler in self.device_battery_logger.handlers:
            handler.close()
            self.device_battery_logger.removeHandler(handler)

        print("SmartHome system shut down.")

if __name__ == "__main__":
    print("Initializing SmartHome...")

    testHome = SmartHome(threshold=11.0, ideal_temp=22, simulate=False)
    print("SmartHome Initialized.")

    try:
        lamp = testHome.create_device("Lamp", DeviceTypeEnum.DECORATIVE, DeviceGroupEnum.LIGHTS, DeviceLocationEnum.LIVINGROOM, is_turned_on=True, battery_level=100, power_consumption=1.5, max_battery_capacity=11111)
        fan = testHome.create_device("Fan", DeviceTypeEnum.DECORATIVE, DeviceGroupEnum.LIGHTS, DeviceLocationEnum.LIVINGROOM, is_turned_on=True, battery_level=100, power_consumption=1.5, max_battery_capacity=111111)

        ac = testHome.create_device("AC", DeviceTypeEnum.DECORATIVE, DeviceGroupEnum.AIRCONDITIONERS, DeviceLocationEnum.LIVINGROOM, is_turned_on=True, power_consumption=1.0, max_battery_capacity=1111111, power_level=1)

        testHome.add_device(lamp)
        testHome.add_device(fan)
        testHome.add_device(ac)

        print("\nDevices added.")
        print("Powered On:", [d.get_device_name() for d in testHome.get_powered_on_devices()])
        print("Powered Off:", [d.get_device_name() for d in testHome.get_powered_off_devices()])
        print(f"Initial Power Consumption: {testHome.get_power_consumption():.2f}W")

        print("\nAdding a person to Living Room...")
        testHome.add_person(DeviceLocationEnum.LIVINGROOM)

        time.sleep(0.1)
        print(f"People in Living Room: {testHome.get_location(DeviceLocationEnum.LIVINGROOM.name).get_people()}")

        print("\nAdding a rule to turn off Lamp...")
        try:
            rule = testHome.parse_rule("turn Lamp off")
            testHome.add_rule(rule)
        except RuleParsingException as e:
            print(f"Rule parsing failed: {e}")

        print("\nRunning simulation for 10 seconds...")
        time.sleep(5)

        print("\n--- Final State ---")
        print("Powered On:", [d.get_device_name() for d in testHome.get_powered_on_devices()])
        print("Powered Off:", [d.get_device_name() for d in testHome.get_powered_off_devices()])
        print(f"Final Power Consumption: {testHome.get_power_consumption():.2f}W")

        print("\n--- Recent Logs (Last 10) ---")
        print("Info:", testHome.get_info_tasks()[-10:])
        print("Warnings:", testHome.get_warning_tasks()[-10:])
        print("Errors/Critical:", testHome.get_severe_tasks()[-10:])
        print("Power:", testHome.get_power_consumption_tasks()[-10:])
        print("Battery:", testHome.get_device_battery_tasks()[-10:])

    except Exception as main_e:
        print(f"\nAn error occurred during the example execution: {main_e}")
        logging.exception("Main execution error") 

    finally:

        print("\nShutting down SmartHome...")
        testHome.shutdown()
        print("Shutdown complete.")

Initializing SmartHome...
SmartHome Initialized.
ENQUEING TASK
ENQUEING TASK
ENQUEING TASK

Devices added.
Powered On: ['Lamp', 'Fan', 'AC']
Powered Off: []
Initial Power Consumption: 0.00W

Adding a person to Living Room...
People in Living Room: 1

Adding a rule to turn off Lamp...

Running simulation for 10 seconds...
Processing device with ID: 1
Executing turn_off for device: Lamp
Processing device with ID: 1
Executing set_power_level for device: Fan
Setting power level to: 5
Processing device with ID: 1
Executing set_power_level for device: Fan
Setting power level to: 5
Processing device with ID: 1
Executing set_power_level for device: Fan
Setting power level to: 5
Processing device with ID: 1
Executing set_power_level for device: Fan
Setting power level to: 5
Processing device with ID: 1
Executing set_power_level for device: Fan
Setting power level to: 5

--- Final State ---
Powered On: ['Fan', 'AC']
Powered Off: ['Lamp']
Final Power Consumption: 8.25W

--- Recent Logs (Last 10) 

## User Interface

The user interface provides a user-friendly way to interact with the smart home system:

*   **User-Friendly Access:**  A visual interface for easy interaction.
*   **Device Status Display:** Shows the real-time status of devices (on/off, power level, battery level).
*   **Device Control:** Allows users to manually control devices (e.g., toggle on/off).
*   **Log Monitoring:**  Allows users to monitor all logs

In [23]:
from flask import Flask, request, jsonify
from flask_cors import CORS, cross_origin
import json

app = Flask("Smart_Home_Automation")
CORS(app, supports_credentials=False, origins="*")
app.config['Headers'] = 'Access-Control-Allow-Origin'
app.config['CORS_HEADERS'] = 'Content-Type'
app.config['CONTENT_TYPE'] = 'multipart/form-data'

smart_home = SmartHome(10,25, False)
smart_home.add_device(smart_home.create_device("Lamp", DeviceTypeEnum.DECORATIVE, DeviceGroupEnum.LIGHTS, DeviceLocationEnum.LIVINGROOM, True, 100, 1, 1000, 1))
smart_home.add_device(smart_home.create_device("Fan", DeviceTypeEnum.DECORATIVE, DeviceGroupEnum.LIGHTS, DeviceLocationEnum.LIVINGROOM, True, 100, 2, 1000, 1))
smart_home.add_device(smart_home.create_device("AC", DeviceTypeEnum.DECORATIVE, DeviceGroupEnum.AIRCONDITIONERS, DeviceLocationEnum.LIVINGROOM, True, 100, 3, 1000, 1))

@app.route("/devices", methods=["GET"])
@cross_origin()
def get_all_devices():
    devices = smart_home.get_devices()
    device_list = [device.to_dict() for device in devices if isinstance(device, Device)]
    return jsonify(device_list)

@app.route("/devices/off", methods=["GET"])
@cross_origin()
def get_off_devices():
    devices = smart_home.get_powered_off_devices()
    device_list = [device.to_dict() for device in devices if isinstance(device, Device)]
    return jsonify(device_list)

@app.route("/devices/on", methods=["GET"])
@cross_origin()
def get_on_devices():
    devices = smart_home.get_powered_on_devices()
    device_list = [device.to_dict() for device in devices if isinstance(device, Device)]
    return jsonify(device_list)

@app.route("/devices/id/<id>", methods=["GET"])
@cross_origin()
def get_device_by_id(id):
    try:
        device_id = int(id)
        device = smart_home.get_device_by_id(device_id)
        if device and isinstance(device, Device):
            return jsonify(device.to_dict())
        else:
            status = 404 if not device else 500 
            msg = f"Device with id {device_id} not found" if not device else "Internal error: Invalid device object"
            return jsonify({"error": msg}), status
    except ValueError:
        return jsonify({"error": f"Invalid ID format: {id}"}), 400
    except Exception as e:
        print(f"Error in get_device_by_id: {e}")
        return jsonify({"error": "Internal server error"}), 500

@app.route("/devices/name/<name>", methods=["GET"])
@cross_origin()
def get_device_by_name(name):

    device = smart_home.get_device_by_name(str(name))
    if device and isinstance(device, Device):
        return jsonify(device.to_dict())
    else:
        status = 404 if not device else 500
        msg = f"Device with name '{name}' not found" if not device else "Internal error: Invalid device object"
        return jsonify({"error": msg}), status

@app.route("/devices/power_consumption", methods=["GET"])
@cross_origin()
def get_power_consumption():
    return jsonify(smart_home.get_power_consumption())

@app.route("/devices/threshold", methods=["GET"])
@cross_origin()
def get_power_consumption_threshold():
    return jsonify(smart_home.get_threshold())

@app.route("/devices/ideal_temp", methods=["GET"])
@cross_origin()
def get_ideal_temperature():
    return jsonify(smart_home.get_ideal_temp())

@app.route("/devices/groups", methods=["GET"])
@cross_origin()
def get_device_groups():
    groups = smart_home.get_device_groups()
    return jsonify(list(groups.keys()) if isinstance(groups, dict) else groups)

@app.route("/devices/locations", methods=["GET"])
@cross_origin()
def get_device_locations():
    locations = smart_home.get_device_locations()
    return jsonify(list(locations.keys()) if isinstance(locations, dict) else locations)

@app.route("/devices/types", methods=["GET"])
@cross_origin()
def get_device_types():
    types = smart_home.get_device_types()
    return jsonify(list(types.keys()) if isinstance(types, dict) else types)

@app.route("/devices/groups/devices", methods=["GET"])
@cross_origin()
def get_devices_by_group():
    group = request.args.get("group")
    if not group:
         return jsonify({"error": "Missing 'group' query parameter"}), 400
    devices = smart_home.get_devices_by_group(group)
    device_list = [device.to_dict() for device in devices]
    return jsonify(device_list)

@app.route("/devices/locations/devices", methods=["GET"])
@cross_origin()
def get_devices_by_location():
    location = request.args.get("location")
    if not location:
        return jsonify({"error": "Missing 'location' query parameter"}), 400
    
    devices = smart_home.get_devices_by_location(location)
    device_list = [device.to_dict() for device in devices]
    return jsonify(device_list)

@app.route("/devices/types/devices", methods=["GET"])
@cross_origin()
def get_devices_by_type():
    device_type = request.args.get("type")
    if not device_type:
        return jsonify({"error": "Missing 'type' query parameter"}), 400
    devices = smart_home.get_devices_by_type(device_type)
    device_list = [device.to_dict() for device in devices]
    return jsonify(device_list)

@app.route("/devices/name/<name>/power_level", methods=["GET"])
@cross_origin()
def get_power_level_by_name(name):
    device = smart_home.get_device_by_name(name)
    if device:
        return jsonify(device.get_power_level())
    else:
        return jsonify({"error": f"Device with name '{name}' not found"}), 404

@app.route("/devices/id/<id>/power_level", methods=["GET"])
@cross_origin()
def get_power_level_by_id(id):
    try:
        device_id = int(id)
        device = smart_home.get_device_by_id(device_id)
        if device:
            return jsonify(device.get_power_level())
        else:
            return jsonify({"error": f"Device with id {device_id} not found"}), 404
    except ValueError:
        return jsonify({"error": f"Invalid ID format: {id}"}), 400

@app.route("/devices/name/<name>/battery_level", methods=["GET"])
@cross_origin()
def get_battery_level_by_name(name):
    device = smart_home.get_device_by_name(name)
    if device:
        return jsonify(device.get_battery_level())
    else:
        return jsonify({"error": f"Device with name '{name}' not found"}), 404

@app.route("/devices/id/<id>/battery_level", methods=["GET"])
@cross_origin()
def get_battery_level_by_id(id):
    try:
        device_id = int(id)
        device = smart_home.get_device_by_id(device_id)
        if device:
            return jsonify(device.get_battery_level())
        else:
            return jsonify({"error": f"Device with id {device_id} not found"}), 404
    except ValueError:
        return jsonify({"error": f"Invalid ID format: {id}"}), 400

@app.route("/devices/name/<name>/max_battery_capacity", methods=["GET"])
@cross_origin()
def get_max_battery_capacity_by_name(name):
    device = smart_home.get_device_by_name(name)
    if device:

        return jsonify(device.get_battery_capacity())
    else:
        return jsonify({"error": f"Device with name '{name}' not found"}), 404

@app.route("/devices/id/<id>/max_battery_capacity", methods=["GET"])
@cross_origin()
def get_max_battery_capacity_by_id(id):
    try:
        device_id = int(id)
        device = smart_home.get_device_by_id(device_id)
        if device:
            return jsonify(device.get_battery_capacity())
        else:
            return jsonify({"error": f"Device with id {device_id} not found"}), 404
    except ValueError:
        return jsonify({"error": f"Invalid ID format: {id}"}), 400

@app.route("/devices/name/<name>/power_consumption", methods=["GET"])
@cross_origin()
def get_power_consumption_by_name(name):
    device = smart_home.get_device_by_name(name)
    if device:

        return jsonify(device.get_base_power_consumption())
    else:
        return jsonify({"error": f"Device with name '{name}' not found"}), 404

@app.route("/devices/id/<id>/power_consumption", methods=["GET"])
@cross_origin()
def get_power_consumption_by_id(id):
    try:
        device_id = int(id)
        device = smart_home.get_device_by_id(device_id)
        if device:
            return jsonify(device.get_base_power_consumption())
        else:
            return jsonify({"error": f"Device with id {device_id} not found"}), 404
    except ValueError:
        return jsonify({"error": f"Invalid ID format: {id}"}), 400

@app.route("/devices/log/info", methods=["GET"])
@cross_origin()
def get_info_logs():
    logs = list(smart_home.get_info_tasks()) 
    smart_home.clear_info_tasks()

    return jsonify(convert_log_task_list(logs))

@app.route("/devices/log/warning", methods=["GET"])
@cross_origin()
def get_warning_logs():
    logs = list(smart_home.get_warning_tasks())
    smart_home.clear_warning_tasks()
    return jsonify(convert_log_task_list(logs))

@app.route("/devices/log/severe", methods=["GET"])
@cross_origin()
def get_error_logs():
    logs = list(smart_home.get_severe_tasks())
    smart_home.clear_severe_tasks()
    return jsonify(convert_log_task_list(logs))

@app.route("/devices/log/power_consumption", methods=["GET"])
@cross_origin()
def get_power_consumption_logs():
    logs = list(smart_home.get_power_consumption_tasks()) 
    smart_home.clear_power_consumption_tasks()
    return jsonify(convert_log_task_list(logs)) 

@app.route("/devices/log/battery", methods=["GET"])
@cross_origin()
def get_battery_logs():
    logs = list(smart_home.get_device_battery_tasks()) 
    smart_home.clear_device_battery_tasks()
    return jsonify(convert_log_task_list(logs))

@app.route("/devices/debug/linkedlists", methods=["GET"])
@cross_origin()
def get_linked_lists():

    try:
        logging_list_data = convert_log_task_list(smart_home.get_logging_list())
        power_log_data = convert_log_task_list(smart_home.get_power_consumption_log_list())
        battery_log_data = convert_log_task_list(smart_home.get_device_battery_log_list())
        rule_list_data = convert_rule_list(smart_home.get_rule_list())

        linked_lists_data = {
            "loggingList": logging_list_data,
            "powerConsumptionLogList": power_log_data,
            "deviceBatteryLogList": battery_log_data,
            "ruleList": rule_list_data
        }
        return jsonify(linked_lists_data)
    except Exception as e:
        print(f"Error in /debug/linkedlists: {e}")

        return jsonify({"error": "Failed to serialize linked list data"}), 500

@app.route("/devices/debug/priorityqueues", methods=["GET"])
@cross_origin()
def get_priority_queues():

    try:
        priority_queues_data = {
            "deviceQueue": convert_device_queue(smart_home.get_device_queue()),
            "powerReducibleDevices": convert_device_queue(smart_home.get_power_reducible_devices()),
            "turnBackOnDevices": convert_device_queue(smart_home.get_turn_back_on_devices())
        }
        return jsonify(priority_queues_data)
    except Exception as e:
        print(f"Error in /debug/priorityqueues: {e}")

        return jsonify({"error": "Failed to serialize priority queue data"}), 500

def convert_log_task_list(log_source):
    """
    Converts a list, tuple, or LinkedList of LogTask objects
    (or strings) into a list of dictionaries.
    """
    converted_list = []
    items_to_process = []

    if isinstance(log_source, (list, tuple)):
        items_to_process = log_source
    elif isinstance(log_source, LinkedList):

        with log_source.lock:
            items_to_process = log_source._make_list()
    else:
        print(f"Warning: convert_log_task_list received incompatible type: {type(log_source)}")
        return []

    for item in items_to_process:
        if isinstance(item, LogTask):

            if hasattr(item, 'to_dict') and callable(item.to_dict):
                converted_list.append(item.to_dict())
            else:
                print(f"Warning: LogTask object lacks to_dict method: {item}")
                converted_list.append({"error": "LogTask missing to_dict"})

        elif isinstance(item, str):
            converted_list.append(item) 
        else:

            print(f"Warning: Item in log source is not a LogTask or str: {type(item)}")
            converted_list.append({"error": f"Invalid log item type: {type(item)}"})

    return converted_list

def convert_rule_list(rule_source):
    """
    Converts a list, tuple, or LinkedList of Rule objects
    into a list of dictionaries.
    """
    converted_list = []
    items_to_process = []

    if isinstance(rule_source, (list, tuple)):
        items_to_process = rule_source
    elif isinstance(rule_source, LinkedList):

        with rule_source.lock:
             items_to_process = rule_source._make_list()
    else:
        print(f"Warning: convert_rule_list received incompatible type: {type(rule_source)}")
        return []

    for item in items_to_process:
        if isinstance(item, Rule):

            if hasattr(item, 'to_dict') and callable(item.to_dict):
                converted_list.append(item.to_dict())
            else:
                 print(f"Warning: Rule object lacks to_dict method: {item}")
                 converted_list.append({"error": "Rule missing to_dict"})
        else:

            print(f"Warning: Item in rule source is not a Rule object: {type(item)}")

            converted_list.append({"error": f"Invalid rule item type: {type(item)}"})

    return converted_list

def convert_device_queue(queue_obj):
    """
    Converts a PriorityQueue containing Task objects into a list of
    dictionaries, where each dict represents a Task's content and priority.
    Assumes Task objects have 'task' and 'priority' attributes.
    The 'task' attribute might be a Device object, str, int, etc.
    """
    converted_list = []

    if not isinstance(queue_obj, PriorityQueue):
        print(f"Warning: convert_device_queue received non-PriorityQueue object: {type(queue_obj)}")

        return []

    try:
        items_in_queue = queue_obj.queue._make_list()

        for item in items_in_queue:

            if isinstance(item, Task):
                task_content = item.task 
                priority = item.priority 

                name = "Unknown"
                if isinstance(task_content, Device):

                    if hasattr(task_content, 'get_device_name') and callable(task_content.get_device_name):
                         name = task_content.get_device_name()
                    else:
                         print(f"Warning: Device object lacks get_device_name method: {task_content}")
                         name = f"Device (no name method: {type(task_content)})"
                elif isinstance(task_content, (str, int)):
                    name = str(task_content) 
                else:

                    name = f"Object ({type(task_content)})"

                map_obj = {
                    "Name": name,       
                    "Priority": priority
                }
                converted_list.append(map_obj)
            else:

                print(f"Warning: Item in device queue is not a Task object: {type(item)}")
                converted_list.append({"error": f"Invalid queue item type: {type(item)}"})

    except Exception as e:
        print(f"Error during convert_device_queue: {e}")

    return converted_list


@app.route("/devices", methods=["POST"])
@cross_origin()
def add_device():
    if not request.is_json:
        return jsonify({"error": "Request must be JSON"}), 400

    device_data = request.json

    try:

        device_type = DeviceTypeEnum[device_data['device_type']]
        device_group = DeviceGroupEnum[device_data['device_group']]
        location = DeviceLocationEnum[device_data['location']]

        device = smart_home.create_device(
            device_data['device_name'],
            device_type,
            device_group,
            location,
            device_data.get('is_turned_on', False),
            device_data.get('battery_level', 100.0),
            device_data.get('base_power_consumption', 0.0),
            device_data.get('max_battery_capacity', 0),
            device_data.get('power_level', 0),
        )

        smart_home.add_device(device)

        return jsonify({
            "message": "Device added successfully",
            "device": device.to_dict()
        }), 201

    except (KeyError, ValueError) as e:
        return jsonify({"error": f"Invalid device data: {str(e)}"}), 400

@app.route("/devices/id/<id>/on", methods=["PUT"])
@cross_origin()
def turn_on_device_by_id(id):
    try:
        device_id = int(id)
        device = smart_home.get_device_by_id(device_id)
        if device:
            smart_home.turn_on_device(device)
            return jsonify({"message": f"Device '{device.get_device_name()}' turned on successfully"})
        else:
            return jsonify({"error": f"Device with id {device_id} not found"}), 404
    except ValueError:
        return jsonify({"error": f"Invalid ID format: {id}"}), 400

@app.route("/devices/name/<name>/on", methods=["PUT"])
@cross_origin()
def turn_on_device_by_name(name):
    device = smart_home.get_device_by_name(name)
    if device:
        smart_home.turn_on_device(device)
        return jsonify({"message": f"Device '{name}' turned on successfully"})
    else:
        return jsonify({"error": f"Device with name '{name}' not found"}), 404

@app.route("/devices/id/<id>/off", methods=["PUT"])
@cross_origin()
def turn_off_device_by_id(id):
    try:
        device_id = int(id)
        device = smart_home.get_device_by_id(device_id)
        if device:
            smart_home.turn_off_device(device)
            return jsonify({"message": f"Device '{device.get_device_name()}' turned off successfully"})
        else:
            return jsonify({"error": f"Device with id {device_id} not found"}), 404
    except ValueError:
        return jsonify({"error": f"Invalid ID format: {id}"}), 400

@app.route("/devices/name/<name>/off", methods=["PUT"])
@cross_origin()
def turn_off_device_by_name(name):
    device = smart_home.get_device_by_name(name)
    if device:
        smart_home.turn_off_device(device)
        return jsonify({"message": f"Device '{name}' turned off successfully"})
    else:
        return jsonify({"error": f"Device with name '{name}' not found"}), 404

@app.route("/devices/threshold/<threshold>", methods=["PUT"])
@cross_origin()
def set_power_consumption_threshold(threshold):
    try:
        threshold_value = float(threshold)
        smart_home.set_threshold(threshold_value)
        return jsonify({"message": f"Threshold set successfully to {threshold_value}"})
    except ValueError:
        return jsonify({"error": f"Invalid threshold format: {threshold}. Must be a number."}), 400

@app.route("/devices/ideal_temp/<ideal_temp>", methods=["PUT"])
@cross_origin()
def set_ideal_temperature(ideal_temp):
    try:
        temp_value = int(ideal_temp)
        smart_home.set_ideal_temp(temp_value)
        return jsonify({"message": f"Ideal temperature set successfully to {temp_value}"})
    except ValueError:
         return jsonify({"error": f"Invalid ideal temperature format: {ideal_temp}. Must be an integer."}), 400

@app.route("/devices/id/<id>/power_level/<power_level>", methods=["PUT"])
@cross_origin()
def set_power_level_by_id(id, power_level):
    rule_string = f"set {id} {power_level}"
    rule = smart_home.parse_rule(rule_string)
    if rule:

         device_exists = smart_home.get_device_by_id(rule.get('device_id'))
         if device_exists:
              smart_home.add_rule(rule) 
              return jsonify({"message": f"Power level rule added/executed for device ID {id}"})
         else:
              return jsonify({"error": f"Device with ID {id} not found"}), 404
    else:
         return jsonify({"error": f"Failed to parse rule or invalid power level: {rule_string}"}), 400

@app.route("/devices/name/<name>/power_level/<power_level>", methods=["PUT"])
@cross_origin()
def set_power_level_by_name(name, power_level):
    rule_string = f"set {name} {power_level}"
    rule = smart_home.parse_rule(rule_string)
    if rule:

         device_exists = smart_home.get_device_by_name(rule.get('device_name'))
         if device_exists:
              smart_home.add_rule(rule) 
              return jsonify({"message": f"Power level rule added/executed for device '{name}'"})
         else:
              return jsonify({"error": f"Device with name '{name}' not found"}), 404
    else:
        return jsonify({"error": f"Failed to parse rule or invalid power level: {rule_string}"}), 400

@app.route("/devices/location/<location>/add_person", methods=["PUT"])
@cross_origin()
def add_person_to_location(location):

    loc_obj = smart_home.get_location(location)
    if loc_obj:
        smart_home.add_person(loc_obj.location)
        return jsonify({"message": f"Person added successfully to location '{location}'"})
    else:
        return jsonify({"error": f"Location '{location}' not found or invalid"}), 404

@app.route("/devices/location/<location>/remove_person", methods=["PUT"])
@cross_origin()
def remove_person_from_location(location):
    loc_obj = smart_home.get_location(location)
    if loc_obj:
        smart_home.remove_person(loc_obj.location)
        return jsonify({"message": f"Person removed successfully from location '{location}'"})
    else:
        return jsonify({"error": f"Location '{location}' not found or invalid"}), 404

@app.route("/devices/name/<name>", methods=["DELETE"])
@cross_origin()
def remove_device_by_name(name):
    device = smart_home.get_device_by_name(name)
    if device:
        if smart_home.remove_device(device):
            return jsonify({"message": f"Device '{name}' removed successfully"})
        else:

            return jsonify({"error": f"Failed to remove device '{name}'"}), 500
    else:
        return jsonify({"error": f"Device with name '{name}' not found"}), 404

@app.route("/devices/id/<id>", methods=["DELETE"])
@cross_origin()
def remove_device_by_id(id):
    try:
        device_id = int(id)
        device = smart_home.get_device_by_id(device_id)
        if device:
            if smart_home.remove_device(device):
                return jsonify({"message": f"Device with ID {device_id} removed successfully"})
            else:
                return jsonify({"error": f"Failed to remove device with ID {device_id}"}), 500
        else:
            return jsonify({"error": f"Device with id {device_id} not found"}), 404
    except ValueError:
        return jsonify({"error": f"Invalid ID format: {id}"}), 400

app.run(host='127.0.0.1', port=8080)

ENQUEING TASK
ENQUEING TASK
ENQUEING TASK
 * Serving Flask app 'Smart_Home_Automation'
 * Debug mode: off


2025-04-14 19:42:24,427 - INFO - WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8080
2025-04-14 19:42:24,427 - INFO - Press CTRL+C to quit


Processing device with ID: 3
Executing set_power_level for device: AC
Setting power level to: 4
Processing device with ID: 3
Executing set_power_level for device: AC
Setting power level to: 4
Processing device with ID: 3
Executing set_power_level for device: AC
Setting power level to: 4
Processing device with ID: 3
Executing set_power_level for device: AC
Setting power level to: 1
Processing device with ID: 3
Executing set_power_level for device: AC
Setting power level to: 4
Processing device with ID: 3
Executing set_power_level for device: AC
Setting power level to: 4
Processing device with ID: 3
Executing turn_off for device: AC
Processing device with ID: 1
Executing turn_on for device: Lamp


2025-04-14 19:42:38,195 - INFO - 127.0.0.1 - - [14/Apr/2025 19:42:38] "GET /devices HTTP/1.1" 200 -
2025-04-14 19:42:38,198 - INFO - 127.0.0.1 - - [14/Apr/2025 19:42:38] "GET /devices/log/info HTTP/1.1" 200 -
2025-04-14 19:42:38,198 - INFO - 127.0.0.1 - - [14/Apr/2025 19:42:38] "GET /devices/debug/linkedlists HTTP/1.1" 200 -
2025-04-14 19:42:38,201 - INFO - 127.0.0.1 - - [14/Apr/2025 19:42:38] "GET /devices/groups HTTP/1.1" 200 -
2025-04-14 19:42:38,460 - INFO - 127.0.0.1 - - [14/Apr/2025 19:42:38] "GET /devices/log/warning HTTP/1.1" 200 -
2025-04-14 19:42:38,524 - INFO - 127.0.0.1 - - [14/Apr/2025 19:42:38] "GET /devices/debug/priorityqueues HTTP/1.1" 200 -
2025-04-14 19:42:38,527 - INFO - 127.0.0.1 - - [14/Apr/2025 19:42:38] "GET /devices/locations HTTP/1.1" 200 -
2025-04-14 19:42:38,529 - INFO - 127.0.0.1 - - [14/Apr/2025 19:42:38] "GET /devices/log/severe HTTP/1.1" 200 -
2025-04-14 19:42:38,770 - INFO - 127.0.0.1 - - [14/Apr/2025 19:42:38] "GET /devices/locations HTTP/1.1" 200 -
20

ENQUEING TASK


...

2025-04-14 19:44:07,404 - INFO - 127.0.0.1 - - [14/Apr/2025 19:44:07] "GET /devices HTTP/1.1" 200 -
2025-04-14 19:44:08,085 - INFO - 127.0.0.1 - - [14/Apr/2025 19:44:08] "GET /devices HTTP/1.1" 200 -
2025-04-14 19:44:09,411 - INFO - 127.0.0.1 - - [14/Apr/2025 19:44:09] "GET /devices/log/info HTTP/1.1" 200 -
2025-04-14 19:44:09,414 - INFO - 127.0.0.1 - - [14/Apr/2025 19:44:09] "GET /devices HTTP/1.1" 200 -
2025-04-14 19:44:09,416 - INFO - 127.0.0.1 - - [14/Apr/2025 19:44:09] "GET /devices/debug/linkedlists HTTP/1.1" 200 -
2025-04-14 19:44:09,674 - INFO - 127.0.0.1 - - [14/Apr/2025 19:44:09] "GET /devices/log/warning HTTP/1.1" 200 -
2025-04-14 19:44:09,734 - INFO - 127.0.0.1 - - [14/Apr/2025 19:44:09] "GET /devices/debug/priorityqueues HTTP/1.1" 200 -
2025-04-14 19:44:09,737 - INFO - 127.0.0.1 - - [14/Apr/2025 19:44:09] "GET /devices/log/severe HTTP/1.1" 200 -
2025-04-14 19:44:09,984 - INFO - 127.0.0.1 - - [14/Apr/2025 19:44:09] "GET /devices/log/battery HTTP/1.1" 200 -
2025-04-14 19:44:

Processing device with ID: 2
Executing turn_on for device: Fan
ENQUEING TASK


2025-04-14 19:44:13,090 - INFO - 127.0.0.1 - - [14/Apr/2025 19:44:13] "GET /devices HTTP/1.1" 200 -


Processing device with ID: 1
Executing turn_on for device: Lamp
ENQUEING TASK


2025-04-14 19:44:14,402 - INFO - 127.0.0.1 - - [14/Apr/2025 19:44:14] "GET /devices/log/info HTTP/1.1" 200 -
2025-04-14 19:44:14,405 - INFO - 127.0.0.1 - - [14/Apr/2025 19:44:14] "GET /devices HTTP/1.1" 200 -
2025-04-14 19:44:14,407 - INFO - 127.0.0.1 - - [14/Apr/2025 19:44:14] "GET /devices/types HTTP/1.1" 200 -
2025-04-14 19:44:14,412 - INFO - 127.0.0.1 - - [14/Apr/2025 19:44:14] "GET /devices/debug/linkedlists HTTP/1.1" 200 -
2025-04-14 19:44:14,405 - INFO - 127.0.0.1 - - [14/Apr/2025 19:44:14] "GET /devices HTTP/1.1" 200 -
2025-04-14 19:44:14,407 - INFO - 127.0.0.1 - - [14/Apr/2025 19:44:14] "GET /devices/types HTTP/1.1" 200 -
2025-04-14 19:44:14,412 - INFO - 127.0.0.1 - - [14/Apr/2025 19:44:14] "GET /devices/debug/linkedlists HTTP/1.1" 200 -
2025-04-14 19:44:14,667 - INFO - 127.0.0.1 - - [14/Apr/2025 19:44:14] "GET /devices HTTP/1.1" 200 -
2025-04-14 19:44:14,714 - INFO - 127.0.0.1 - - [14/Apr/2025 19:44:14] "GET /devices/log/warning HTTP/1.1" 200 -
2025-04-14 19:44:14,718 - INFO 